In [1]:
import os
os.chdir("C:/Users/ibigirimana/OneDrive - lallemand.com/Bureau/API_LALLEMAND_Formulaire_Complete")
print("Répertoire courant :", os.getcwd())

Répertoire courant : C:\Users\ibigirimana\OneDrive - lallemand.com\Bureau\API_LALLEMAND_Formulaire_Complete


# *database.py*

In [4]:
%%writefile database.py

"""Database configuration"""
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy.engine import URL
import logging

# Configuration du logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Paramètres de connexion (peuvent être mis dans des variables d'environnement)
SERVER = os.getenv('DB_SERVER', 'localhost\\SQLEXPRESS')
DATABASE = os.getenv('DB_NAME', 'Lallemand')
DRIVER = os.getenv('DB_DRIVER', 'ODBC Driver 17 for SQL Server')

# Construction de l'URL de connexion SQLAlchemy
connection_url = URL.create(
    "mssql+pyodbc",
    host=SERVER,
    database=DATABASE,
    query={
        "driver": DRIVER,
        "trusted_connection": "yes",
        "encrypt": "no",  # Peut être ajusté selon vos besoins de sécurité
    }
)

# Créer le moteur de base de données
engine = create_engine(
    connection_url,
    echo=False,  # Mettre à True pour voir les requêtes SQL
    pool_pre_ping=True,  # Vérification automatique des connexions
    pool_recycle=3600,   # Recyclage des connexions après 1h
)

# Configuration des sessions
SessionLocal = sessionmaker(
    autocommit=False,
    autoflush=False,
    bind=engine
)

# Classe de base pour les modèles ORM
Base = declarative_base()

def get_db():
    """Générateur de session de base de données pour dependency injection"""
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

def test_connection():
    """Teste la connexion à la base de données"""
    try:
        with engine.connect() as conn:
            result = conn.execute("SELECT 1 as test")
            logger.info("Connexion à la base de données réussie.")
            return True
    except Exception as e:
        logger.error(f"Erreur de connexion à la base de données : {e}")
        return False

def create_tables():
    """Crée toutes les tables définies dans les modèles"""
    try:
        Base.metadata.create_all(bind=engine)
        logger.info("Tables créées avec succès.")
    except Exception as e:
        logger.error(f"Erreur lors de la création des tables : {e}")

if __name__ == "__main__":
    # Test de la configuration
    if test_connection():
        print("✅ Configuration de base de données validée")
    else:
        print("❌ Problème de configuration de base de données")

Overwriting database.py


# *models.py*

In [6]:
%%writefile models.py

from sqlalchemy import Column, Integer, String, Float, ForeignKey, Boolean, Text
from sqlalchemy.orm import relationship
from database import Base

class Fermentation(Base):
    __tablename__ = "Fermentation"
    __table_args__ = {'extend_existing': True}
    
    Code = Column(String(255), primary_key=True)
    Souche = Column(String(255))
    Milieu = Column(String(255))
    Volume = Column(String(255))
    Csg_T = Column(Integer)
    Type_fermentation = Column(Text)
    
    # Relations
    donnees_fermentation = relationship("DonneesFermentation", back_populates="fermentation", cascade="all, delete-orphan")
    phase_fermentation = relationship("PhaseFermentation", back_populates="fermentation", uselist=False, cascade="all, delete-orphan")
    analyse_vin = relationship("AnalyseVin", back_populates="fermentation", uselist=False, cascade="all, delete-orphan")
    analyse_mout = relationship("AnalyseMout", back_populates="fermentation", uselist=False, cascade="all, delete-orphan")
    table_params = relationship("Compositions", back_populates="fermentation", uselist=False, cascade="all, delete-orphan")


class DonneesFermentation(Base):
    __tablename__ = "Donnees_Fermentation"
    __table_args__ = {'extend_existing': True}
    
    Id = Column(Integer, primary_key=True, index=True, autoincrement=True)
    Temps = Column(Float)
    CO2 = Column(Float)
    V = Column(Float)
    Code = Column(String(255), ForeignKey("Fermentation.Code"))
    
    # Relation
    fermentation = relationship("Fermentation", back_populates="donnees_fermentation")


class PhaseFermentation(Base):
    __tablename__ = "Phase_fermentation"
    __table_args__ = {'extend_existing': True}
    
    Code = Column(String(255), ForeignKey("Fermentation.Code"), primary_key=True)
    FIN_LATENCE = Column(Float)
    Max_V = Column(Float)
    TempsmaxV = Column(Float)
    max_co2 = Column(Float)
    tempsmaxco2 = Column(Float)
    Maxco2_95 = Column(Float)
    Maxco2_90 = Column(Float)
    Maxco2_70 = Column(Float)
    Maxco2_80 = Column(Float)
    tempsmaxco2_95 = Column(Float)
    tempsmaxco2_90 = Column(Float)
    tempsmaxco2_70 = Column(Float)
    tempsmaxco2_80 = Column(Float)
    Fin_fermentation = Column(Float)
    
    # Relation
    fermentation = relationship("Fermentation", back_populates="phase_fermentation")


class AnalyseVin(Base):
    __tablename__ = "Analyse_Vin"
    __table_args__ = {'extend_existing': True}
    
    Code = Column(String(255), ForeignKey("Fermentation.Code"), primary_key=True)
    Glucose_fructose_g_l = Column(String(255))  
    Titre_alcoometrique_vol = Column(String(255))  
    pH = Column(String(255))
    Acidite_volatile_gH2SO4_l = Column(String(255)) 
    Acidite_totale_gH2SO4_l = Column(String(255))   
    Dioxyde_soufre_Libre_mg_l = Column(String(255)) 
    Dioxyde_soufre_Total_mg_l = Column(String(255)) 
    AcideL_malique_g_L = Column(String(255))
    Nuance_IR = Column(String(255))
    IC = Column(String(255))
    
    # Relation
    fermentation = relationship("Fermentation", back_populates="analyse_vin")


class AnalyseMout(Base):
    __tablename__ = "Analyse_Mout"
    __table_args__ = {'extend_existing': True}
    
    Code = Column(String(255), ForeignKey("Fermentation.Code"), primary_key=True)
    Synonymes = Column(String(255))
    Code_Milieu = Column(String(255))
    Sucres = Column(String(255))
    Azote = Column(String(255))
    pH = Column(String(255))
    Turbidite = Column(String(255))
    Acide_Malic = Column(String(255))
    Phytosterols_mg_L = Column(String(255)) 
    Ergosterol_mg_L = Column(String(255))    
    SO2_libre = Column(String(255))
    SO2_Total = Column(String(255))
    Desaere = Column(Boolean)         
    AT = Column(String(255))        
    phyt_NTU = Column(String(255))
    ergo_NTU = Column(String(255))
    Turb_phyto = Column(String(255))
    
    # Relation
    fermentation = relationship("Fermentation", back_populates="analyse_mout")


class Compositions(Base):
    __tablename__ = "Compositions"
    __table_args__ = {'extend_existing': True}
    
    Code = Column(String(255), ForeignKey("Fermentation.Code"), primary_key=True)
    Code_Souche = Column(String(255))
    modelisation = Column(Boolean) 
    protectant = Column(Boolean)
    T0_nut_compl = Column(Boolean) 
    T0_nut_org = Column(Boolean)
    un_tiers_FA_nut_comp = Column(Boolean)  
    un_tiers_FA_nut_org = Column(Boolean)   
    NSc = Column(Boolean)
    NSC_sc_seq = Column(Boolean)  
    NSC_tps_seq = Column(Boolean) 
    
    # Relation
    fermentation = relationship("Fermentation", back_populates="table_params")

Overwriting models.py


# *TEST models.py et database.py*

In [7]:
from database import SessionLocal
from models import (
    Fermentation, 
    DonneesFermentation, 
    PhaseFermentation, 
    AnalyseVin, 
    AnalyseMout, 
    Compositions
)
db = SessionLocal()

# *Table Fermentation*

In [8]:
fermentations = db.query(Fermentation).limit(10).all()
if fermentations:
    for fermentation in fermentations:
        print(f"CODE : {fermentation.Code}, Souche : {fermentation.Souche}, Milieu : {fermentation.Milieu}, Volume : {fermentation.Volume}, Csg_T : {fermentation.Csg_T}, Type_fermentation : {fermentation.Type_fermentation}")
else:
    print("No fermentations found.")

CODE : A1_2020-08-05, Souche : Persy, Milieu : MSN250 S270 P20, Volume : Robot, Csg_T : 24, Type_fermentation : Valide
CODE : A1_2020-09-25, Souche : test, Milieu : test, Volume : Robot, Csg_T : 24, Type_fermentation : Non_Valide
CODE : A1_2021-03-03, Souche : 1410, Milieu : MSN180 S240 P3, Volume : Robot, Csg_T : 20, Type_fermentation : Valide
CODE : A1_2021-09-23, Souche : test, Milieu : testt, Volume : Robot, Csg_T : 24, Type_fermentation : Non_Valide
CODE : A1_2021-09-27, Souche : test, Milieu : eau, Volume : Robot, Csg_T : 27, Type_fermentation : Non_Valide
CODE : A1_2021-09-28, Souche : test, Milieu : testt, Volume : Robot, Csg_T : 27, Type_fermentation : Non_Valide
CODE : A1_2022-01-11, Souche : OT69 A4, Milieu : MSN100 S180 P3, Volume : Robot, Csg_T : 20, Type_fermentation : Valide
CODE : A1_2022-02-15, Souche : Laktia, Milieu : Syrah PR20, Volume : Robot, Csg_T : 24, Type_fermentation : Valide
CODE : A1_2022-03-28, Souche : CY3079, Milieu : Maccabeu PR20, Volume : Robot, Csg_T

# *Table DonneesFermentation*

In [9]:
donnees = db.query(DonneesFermentation).limit(10).all()
if donnees:
    for donnee in donnees:
        print(f"ID : {donnee.Id}, Temps : {donnee.Temps}, CO2 : {donnee.CO2}, V : {donnee.V}, Code : {donnee.Code}")
else:
    print("No donnees found.")

ID : 1, Temps : 0.0, CO2 : 0.0, V : 0.0, Code : F10_2013-04-03
ID : 2, Temps : 0.33, CO2 : -0.018, V : 0.0, Code : F10_2013-04-03
ID : 3, Temps : 0.67, CO2 : 0.0, V : -0.109, Code : F10_2013-04-03
ID : 4, Temps : 1.0, CO2 : 0.0, V : -0.035, Code : F10_2013-04-03
ID : 5, Temps : 1.33, CO2 : -0.018, V : 0.01, Code : F10_2013-04-03
ID : 6, Temps : 1.67, CO2 : -0.036, V : 0.023, Code : F10_2013-04-03
ID : 7, Temps : 2.0, CO2 : 0.036, V : -0.028, Code : F10_2013-04-03
ID : 8, Temps : 2.33, CO2 : 0.018, V : -0.009, Code : F10_2013-04-03
ID : 9, Temps : 2.67, CO2 : 0.036, V : 0.003, Code : F10_2013-04-03
ID : 10, Temps : 3.0, CO2 : 0.018, V : 0.011, Code : F10_2013-04-03


# *Table PhaseFermentation*

In [10]:
phases = db.query(PhaseFermentation).limit(100).all()
if phases:
    for phase in phases:
        print(f"Code : {phase.Code}, FIN_LATENCE : {phase.FIN_LATENCE}, Max_V : {phase.Max_V}, TempsmaxV : {phase.TempsmaxV}, max_co2 : {phase.max_co2}, tempsmaxco2 : {phase.tempsmaxco2}, Fin_fermentation : {phase.Fin_fermentation}")
else:
    print("No phases found.")

Code : A1_2020-08-05, FIN_LATENCE : 14.7756002843651, Max_V : 1.77, TempsmaxV : 26.65, max_co2 : 69.49, tempsmaxco2 : 256.68, Fin_fermentation : 182.682150922093
Code : A1_2021-03-03, FIN_LATENCE : 16.3752931997087, Max_V : 1.47, TempsmaxV : 32.19, max_co2 : 115.7, tempsmaxco2 : 364.75, Fin_fermentation : 301.090682243432
Code : A1_2022-01-11, FIN_LATENCE : 13.2137811901633, Max_V : 1.07, TempsmaxV : 29.33, max_co2 : 117.55, tempsmaxco2 : 639.24, Fin_fermentation : 629.177577275707
Code : A1_2022-02-15, FIN_LATENCE : 35.7912515514718, Max_V : 0.91, TempsmaxV : 59.22, max_co2 : 104.24, tempsmaxco2 : 475.92, Fin_fermentation : 477.201105594499
Code : A1_2022-04-13, FIN_LATENCE : 57.4202961780308, Max_V : 1.72, TempsmaxV : 67.94, max_co2 : 124.05, tempsmaxco2 : 472.21, Fin_fermentation : 335.633584282864
Code : A1_2022-11-03, FIN_LATENCE : 6.5631668271696, Max_V : 1.65, TempsmaxV : 18.11, max_co2 : 99.25, tempsmaxco2 : 120.2, Fin_fermentation : 167.351281875095
Code : A1_2022-11-15, FIN_L

# *Table AnalyseVin*

In [11]:
analyses_vin = db.query(AnalyseVin).limit(10).all()
if analyses_vin:
    for analyse in analyses_vin:
        print(f"Code : {analyse.Code}, Glucose_fructose : {analyse.Glucose_fructose_g_l}, Titre_alcool : {analyse.Titre_alcoometrique_vol}, pH : {analyse.pH}, Acidité_volatile : {analyse.Acidite_volatile_gH2SO4_l}, Acidité_totale : {analyse.Acidite_totale_gH2SO4_l}")
else:
    print("No analyses vin found.")

Code : A1_2020-08-05, Glucose_fructose : <0,4, Titre_alcool : 15.78, pH : 3.63, Acidité_volatile : 0.64, Acidité_totale : 7.97
Code : A1_2021-03-03, Glucose_fructose : <0,4, Titre_alcool : 15,04, pH : 3,07, Acidité_volatile : 0,85, Acidité_totale : 9,23
Code : A1_2022-01-11, Glucose_fructose : <0,4, Titre_alcool : 14.72, pH : 3.53, Acidité_volatile : 0.58, Acidité_totale : 7.86
Code : A1_2022-02-15, Glucose_fructose : 1.07, Titre_alcool : 13.59, pH : 3.25, Acidité_volatile : 0.43, Acidité_totale : 6.64
Code : A1_2022-04-13, Glucose_fructose : <0,4, Titre_alcool : 15.38, pH : 3.49, Acidité_volatile : 0.71, Acidité_totale : 7.9
Code : A1_2022-11-03, Glucose_fructose : None, Titre_alcool : None, pH : None, Acidité_volatile : None, Acidité_totale : None
Code : A1_2022-11-15, Glucose_fructose : 1.95, Titre_alcool : 14.26, pH : 3.37, Acidité_volatile : 0.41, Acidité_totale : 4.64
Code : A1_2023-02-07, Glucose_fructose : <0,4, Titre_alcool : 14.37, pH : 3.46, Acidité_volatile : 0.24, Acidité_

# *Table AnalyseMout*

In [12]:
analyses_mout = db.query(AnalyseMout).limit(10).all()
if analyses_mout:
    for analyse in analyses_mout:
        print(f"Code : {analyse.Code}, Synonymes : {analyse.Synonymes}, Code_Milieu : {analyse.Code_Milieu}, Sucres : {analyse.Sucres}, Azote : {analyse.Azote}, pH : {analyse.pH}, Turbidite : {analyse.Turbidite}, Acide_Malic : {analyse.Acide_Malic}")
else:
    print("No analyses mout found.")

Code : A1_2020-08-05, Synonymes : None, Code_Milieu : None, Sucres : None, Azote : None, pH : None, Turbidite : None, Acide_Malic : None
Code : A1_2021-03-03, Synonymes : None, Code_Milieu : None, Sucres : None, Azote : None, pH : None, Turbidite : None, Acide_Malic : None
Code : A1_2022-01-11, Synonymes : None, Code_Milieu : None, Sucres : None, Azote : None, pH : None, Turbidite : None, Acide_Malic : None
Code : A1_2022-02-15, Synonymes : Syrah PR20 (21/06/2021), Code_Milieu : Syrah PR20, Sucres : 239.6, Azote : 144, pH : 3.34, Turbidite : 200, Acide_Malic : None
Code : A1_2022-04-13, Synonymes : None, Code_Milieu : MSN250S260P10S3, Sucres : 260, Azote : 250, pH : 3.3, Turbidite : None, Acide_Malic : None
Code : A1_2022-11-03, Synonymes : None, Code_Milieu : Chardo PR20, Sucres : 208, Azote : 111, pH : 3.42, Turbidite : 50, Acide_Malic : None
Code : A1_2022-11-15, Synonymes : None, Code_Milieu : None, Sucres : None, Azote : None, pH : None, Turbidite : None, Acide_Malic : None
Code :

# *Table Compositions*

In [13]:
parametres = db.query(Compositions).limit(10).all()
if parametres:
    for param in parametres:
        print(f"Code : {param.Code}, Code_Souche : {param.Code_Souche}, modelisation : {param.modelisation}, protectant : {param.protectant}, T0_nut_compl : {param.T0_nut_compl}, T0_nut_org : {param.T0_nut_org}, NSc : {param.NSc}")
else:
    print("No parametres found.")

Code : A1_2020-08-05, Code_Souche : Persy, modelisation : None, protectant : None, T0_nut_compl : None, T0_nut_org : None, NSc : None
Code : A1_2021-03-03, Code_Souche : 1410, modelisation : None, protectant : None, T0_nut_compl : None, T0_nut_org : None, NSc : None
Code : A1_2022-01-11, Code_Souche : OT69 A4, modelisation : None, protectant : None, T0_nut_compl : None, T0_nut_org : None, NSc : None
Code : A1_2022-02-15, Code_Souche : Laktia, modelisation : False, protectant : None, T0_nut_compl : None, T0_nut_org : None, NSc : True
Code : A1_2022-04-13, Code_Souche : DP5, modelisation : None, protectant : None, T0_nut_compl : None, T0_nut_org : None, NSc : None
Code : A1_2022-11-03, Code_Souche : 6D1, modelisation : None, protectant : None, T0_nut_compl : None, T0_nut_org : None, NSc : None
Code : A1_2022-11-15, Code_Souche : 6D1, modelisation : None, protectant : None, T0_nut_compl : None, T0_nut_org : None, NSc : None
Code : A1_2023-02-07, Code_Souche : EC1118, modelisation : True, 

# *query_helper.py*

In [14]:
%%writefile query_helper.py

from sqlalchemy.orm import Session
from sqlalchemy.orm import joinedload
from sqlalchemy import and_
from typing import Optional, List
import models
# Pour les versions récentes de Jupyter
from jupyter_server.serverapp import ServerApp
ServerApp.iopub_data_rate_limit = 100000000
# =============================================================================
# REQUÊTES DE BASE PAR CODE
# =============================================================================
def get_fermentation(db: Session, Code: str):
    """Récupère une fermentation par son Code."""
    return db.query(models.Fermentation).filter(models.Fermentation.Code == Code)

def get_donnees_fermentation(db: Session, Code: str):
    """Récupère les données de fermentation par Code."""
    return db.query(models.DonneesFermentation).filter(models.DonneesFermentation.Code == Code)

def get_phase_fermentation(db: Session, Code: str):
    """Récupère une phase de fermentation par son Code."""
    return db.query(models.PhaseFermentation).filter(models.PhaseFermentation.Code == Code)

def get_analyse_mout(db: Session, Code: str):
    """Récupère une analyse de moût par son Code."""
    return db.query(models.AnalyseMout).filter(models.AnalyseMout.Code == Code)

def get_analyse_vin(db: Session, Code: str):
    """Récupère une analyse de vin par son Code."""
    return db.query(models.AnalyseVin).filter(models.AnalyseVin.Code == Code)

def get_parametres(db: Session, Code: str):
    """Récupère les paramètres par Code."""
    return db.query(models.Compositions).filter(models.Compositions.Code == Code)

# =============================================================================
# REQUÊTE COMPLÈTE PAR SOUCHE ET TEMPÉRATURE - VERSION OPTIMISÉE
# =============================================================================
def get_all_data_by_souche_temperature_dict(db: Session, souche: Optional[str] = None, temperature: Optional[int] = None):
    """
    Récupère toutes les données sous forme de dictionnaire structuré.
    VERSION OPTIMISÉE pour éviter le problème N+1.
    
    Args:
        db: Session de base de données
        souche: Code de la souche (optionnel)
        temperature: Température de consigne (optionnel)
    
    Returns:
        Liste de dictionnaires avec toutes les données organisées
    """
    # 1. D'abord, récupérer toutes les fermentations correspondantes
    query = db.query(models.Fermentation)
    
    # Appliquer les filtres selon les paramètres fournis
    filters = []
    if souche is not None:
        filters.append(models.Fermentation.Souche == souche)
    if temperature is not None:
        filters.append(models.Fermentation.Csg_T == temperature)
    
    # Appliquer les filtres s'il y en a
    if filters:
        query = query.filter(and_(*filters))
    
    fermentations = query.all()
    
    # Si aucune fermentation trouvée, retourner liste vide
    if not fermentations:
        return []
    
    # 2. Extraire tous les codes de fermentation
    codes = [f.Code for f in fermentations]
    
    # 3. Faire une seule requête par table pour tous les codes
    # Récupérer toutes les données de fermentation en une seule requête
    all_donnees = db.query(models.DonneesFermentation).filter(
        models.DonneesFermentation.Code.in_(codes)
    ).all()
    
    # Organiser les données par code
    donnees_by_code = {}
    for donnee in all_donnees:
        if donnee.Code not in donnees_by_code:
            donnees_by_code[donnee.Code] = []
        donnees_by_code[donnee.Code].append(donnee)
    
    # Récupérer toutes les phases en une seule requête
    all_phases = db.query(models.PhaseFermentation).filter(
        models.PhaseFermentation.Code.in_(codes)
    ).all()
    phases_by_code = {phase.Code: phase for phase in all_phases}
    
    # Récupérer toutes les analyses de moût en une seule requête
    all_analyses_mout = db.query(models.AnalyseMout).filter(
        models.AnalyseMout.Code.in_(codes)
    ).all()
    analyses_mout_by_code = {analyse.Code: analyse for analyse in all_analyses_mout}
    
    # Récupérer toutes les analyses de vin en une seule requête
    all_analyses_vin = db.query(models.AnalyseVin).filter(
        models.AnalyseVin.Code.in_(codes)
    ).all()
    analyses_vin_by_code = {analyse.Code: analyse for analyse in all_analyses_vin}
    
    # Récupérer toutes les compositions en une seule requête
    all_compositions = db.query(models.Compositions).filter(
        models.Compositions.Code.in_(codes)
    ).all()
    compositions_by_code = {comp.Code: comp for comp in all_compositions}
    
    # 4. Assembler les résultats
    results = []
    for fermentation in fermentations:
        code = fermentation.Code
        result = {
            'fermentation': fermentation,
            'donnees_fermentation': donnees_by_code.get(code, []),
            'phase_fermentation': phases_by_code.get(code),
            'analyse_mout': analyses_mout_by_code.get(code),
            'analyse_vin': analyses_vin_by_code.get(code),
            'compositions': compositions_by_code.get(code)
        }
        results.append(result)
    
    return results
    

Writing query_helper.py


# *TEST query_helper.py*

In [15]:
from database import SessionLocal
import query_helper as query_helper

# Créer une session
db = SessionLocal()

# *Test get_fermentation*

In [16]:
# Correction 1: Ajouter .first() ou .all() pour exécuter la query
donnees = query_helper.get_fermentation(db, Code="F10_2021-05-07").first()

# Si vous voulez récupérer tous les résultats (au cas où il y en aurait plusieurs) :
# donnees = query_helper.get_fermentation(db, Code="F10_2021-05-07").all()

if donnees:
    # Correction 2: Volume avec majuscule (selon vos modèles)
    print(f"CODE : {donnees.Code}, Souche : {donnees.Souche}, Milieu : {donnees.Milieu}, Volume : {donnees.Volume}, Csg_T : {donnees.Csg_T}, Type_fermentation : {donnees.Type_fermentation}")
else:
    print("No donnes found.")

db.close()

CODE : F10_2021-05-07, Souche : 18-2007, Milieu : chardo PR20, Volume : Salle, Csg_T : 20, Type_fermentation : Valide


# *Test get_donnees_fermentation*

In [17]:
# Test DonneesFermentation
donnees = query_helper.get_donnees_fermentation(db, Code="F10_2021-05-07").first()
# donnees = query_helper.get_donnees_fermentation(db, Code="F10_2021-05-07").all()
if donnees:
    print(f"ID : {donnees.Id}, Temps : {donnees.Temps}, CO2 : {donnees.CO2}, V : {donnees.V}, Code : {donnees.Code}")
else:
    print("No donnees fermentation found.")
db.close() 

ID : 3648932, Temps : 0.0, CO2 : 0.0, V : 0.0, Code : F10_2021-05-07


# *Test get_phase_fermentation*

In [18]:
# Test DonneesFermentation
donnees = query_helper.get_donnees_fermentation(db, Code="F10_2021-05-07").first()
# donnees = query_helper.get_donnees_fermentation(db, Code="F10_2021-05-07").all()
if donnees:
    print(f"ID : {donnees.Id}, Temps : {donnees.Temps}, CO2 : {donnees.CO2}, V : {donnees.V}, Code : {donnees.Code}")
else:
    print("No donnees fermentation found.")
db.close()

ID : 3648932, Temps : 0.0, CO2 : 0.0, V : 0.0, Code : F10_2021-05-07


# *Test get_analyse_mout*

In [19]:
# Test AnalyseMout
donnees = query_helper.get_analyse_mout(db, Code="F10_2021-05-07").first()
# donnees = query_helper.get_analyse_mout(db, Code="F10_2021-05-07").all()
if donnees:
    print(f"Code : {donnees.Code}, Synonymes : {donnees.Synonymes}, Code_Milieu : {donnees.Code_Milieu}, Sucres : {donnees.Sucres}, Azote : {donnees.Azote}, pH : {donnees.pH}, Turbidite : {donnees.Turbidite}, Acide_Malic : {donnees.Acide_Malic}")
else:
    print("No analyse mout found.")
db.close()

Code : F10_2021-05-07, Synonymes : None, Code_Milieu : chardo PR20, Sucres : None, Azote : None, pH : None, Turbidite : None, Acide_Malic : None


# *Test get_analyse_vin*

In [20]:
# Test AnalyseVin
donnees = query_helper.get_analyse_vin(db, Code="F10_2021-05-07").first()
# donnees = query_helper.get_analyse_vin(db, Code="F10_2021-05-07").all()
if donnees:
    print(f"Code : {donnees.Code}, Glucose_fructose : {donnees.Glucose_fructose_g_l}, Titre_alcool : {donnees.Titre_alcoometrique_vol}, pH : {donnees.pH}, Acidité_volatile : {donnees.Acidite_volatile_gH2SO4_l}, Acidité_totale : {donnees.Acidite_totale_gH2SO4_l}")
else:
    print("No analyse vin found.")
db.close()

Code : F10_2021-05-07, Glucose_fructose : 4.75, Titre_alcool : 13.08, pH : 3.38, Acidité_volatile : 0.23, Acidité_totale : 3.88


# *Test get_parametres*

In [21]:
# Test Parametres (Table)
donnees = query_helper.get_parametres(db, Code="F10_2021-05-07").first()
# donnees = query_helper.get_parametres(db, Code="F10_2021-05-07").all()
if donnees:
    print(f"Code : {donnees.Code}, Code_Souche : {donnees.Code_Souche}, modelisation : {donnees.modelisation}, protectant : {donnees.protectant}, T0_nut_compl : {donnees.T0_nut_compl}, T0_nut_org : {donnees.T0_nut_org}, NSc : {donnees.NSc}")
else:
    print("No parametres found.")
db.close()

Code : F10_2021-05-07, Code_Souche : EC1118, modelisation : None, protectant : None, T0_nut_compl : None, T0_nut_org : None, NSc : None


# *get_all_data_by_souche_temperature_dict*

In [22]:
# Code pour afficher TOUTES les données de TOUTES les tables
import query_helper

# Récupérer les données pour une souche et température spécifiques
results = query_helper.get_all_data_by_souche_temperature_dict(db,souche="EC1118",  temperature=16)
print(f"Nombre de fermentations trouvées: {len(results)}")

# Afficher les données de chaque fermentation
for i, result in enumerate(results):
    print(f"\n{'='*80}")
    print(f"FERMENTATION {i+1}")
    print(f"{'='*80}")
    
    # =========================================================================
    # TABLE FERMENTATION - TOUTES LES DONNÉES
    # =========================================================================
    fermentation = result['fermentation']
    print(f"\n📋 TABLE FERMENTATION:")
    print(f"  Code: {fermentation.Code}")
    print(f"  Souche: {fermentation.Souche}")
    print(f"  Milieu: {fermentation.Milieu}")
    print(f"  Volume: {fermentation.Volume}")
    print(f"  Csg_T: {fermentation.Csg_T}")
    print(f"  Type_fermentation: {fermentation.Type_fermentation}")
    
    # =========================================================================
    # TABLE DONNEES_FERMENTATION - TOUS LES POINTS
    # =========================================================================
    donnees_fermentation = result['donnees_fermentation']
    print(f"\n📊 TABLE DONNEES_FERMENTATION:")
    if donnees_fermentation:
        print(f"  Nombre de points: {len(donnees_fermentation)}")
        for j, point in enumerate(donnees_fermentation):
            print(f"    Point {j+1}: Id={point.Id}, Temps={point.Temps}, CO2={point.CO2}, V={point.V}, Code={point.Code}")
    else:
        print("  Aucune donnée de fermentation")
    
    # =========================================================================
    # TABLE PHASE_FERMENTATION - TOUTES LES DONNÉES
    # =========================================================================
    phase = result['phase_fermentation']
    print(f"\n⏱️ TABLE PHASE_FERMENTATION:")
    if phase:
        print(f"  Code: {phase.Code}")
        print(f"  FIN_LATENCE: {phase.FIN_LATENCE}")
        print(f"  Max_V: {phase.Max_V}")
        print(f"  TempsmaxV: {phase.TempsmaxV}")
        print(f"  max_co2: {phase.max_co2}")
        print(f"  tempsmaxco2: {phase.tempsmaxco2}")
        print(f"  Maxco2_95: {phase.Maxco2_95}")
        print(f"  Maxco2_90: {phase.Maxco2_90}")
        print(f"  Maxco2_70: {phase.Maxco2_70}")
        print(f"  Maxco2_80: {phase.Maxco2_80}")
        print(f"  tempsmaxco2_95: {phase.tempsmaxco2_95}")
        print(f"  tempsmaxco2_90: {phase.tempsmaxco2_90}")
        print(f"  tempsmaxco2_70: {phase.tempsmaxco2_70}")
        print(f"  tempsmaxco2_80: {phase.tempsmaxco2_80}")
        print(f"  Fin_fermentation: {phase.Fin_fermentation}")
    else:
        print("  Aucune donnée de phase")
    
    # =========================================================================
    # TABLE ANALYSE_MOUT - TOUTES LES DONNÉES
    # =========================================================================
    analyse_mout = result['analyse_mout']
    print(f"\n🍇 TABLE ANALYSE_MOUT:")
    if analyse_mout:
        print(f"  Code: {analyse_mout.Code}")
        print(f"  Synonymes: {analyse_mout.Synonymes}")
        print(f"  Code_Milieu: {analyse_mout.Code_Milieu}")
        print(f"  Sucres: {analyse_mout.Sucres}")
        print(f"  Azote: {analyse_mout.Azote}")
        print(f"  pH: {analyse_mout.pH}")
        print(f"  Turbidite: {analyse_mout.Turbidite}")
        print(f"  Acide_Malic: {analyse_mout.Acide_Malic}")
        print(f"  Phytosterols_mg_L: {analyse_mout.Phytosterols_mg_L}")
        print(f"  Ergosterol_mg_L: {analyse_mout.Ergosterol_mg_L}")
        print(f"  SO2_libre: {analyse_mout.SO2_libre}")
        print(f"  SO2_Total: {analyse_mout.SO2_Total}")
        print(f"  Desaere: {analyse_mout.Desaere}")
        print(f"  AT: {analyse_mout.AT}")
        print(f"  phyt_NTU: {analyse_mout.phyt_NTU}")
        print(f"  ergo_NTU: {analyse_mout.ergo_NTU}")
        print(f"  Turb_phyto: {analyse_mout.Turb_phyto}")
    else:
        print("  Aucune analyse de moût")
    
    # =========================================================================
    # TABLE ANALYSE_VIN - TOUTES LES DONNÉES
    # =========================================================================
    analyse_vin = result['analyse_vin']
    print(f"\n🍷 TABLE ANALYSE_VIN:")
    if analyse_vin:
        print(f"  Code: {analyse_vin.Code}")
        print(f"  Glucose_fructose_g_l: {analyse_vin.Glucose_fructose_g_l}")
        print(f"  Titre_alcoometrique_vol: {analyse_vin.Titre_alcoometrique_vol}")
        print(f"  pH: {analyse_vin.pH}")
        print(f"  Acidite_volatile_gH2SO4_l: {analyse_vin.Acidite_volatile_gH2SO4_l}")
        print(f"  Acidite_totale_gH2SO4_l: {analyse_vin.Acidite_totale_gH2SO4_l}")
        print(f"  Dioxyde_soufre_Libre_mg_l: {analyse_vin.Dioxyde_soufre_Libre_mg_l}")
        print(f"  Dioxyde_soufre_Total_mg_l: {analyse_vin.Dioxyde_soufre_Total_mg_l}")
        print(f"  AcideL_malique_g_L: {analyse_vin.AcideL_malique_g_L}")
        print(f"  Nuance_IR: {analyse_vin.Nuance_IR}")
        print(f"  IC: {analyse_vin.IC}")
    else:
        print("  Aucune analyse de vin")
    
    # =========================================================================
    # TABLE COMPOSITIONS - TOUTES LES DONNÉES
    # =========================================================================
    compositions = result['compositions']
    print(f"\n🧪 TABLE COMPOSITIONS:")
    if compositions:
        print(f"  Code: {compositions.Code}")
        print(f"  Code_Souche: {compositions.Code_Souche}")
        print(f"  modelisation: {compositions.modelisation}")
        print(f"  protectant: {compositions.protectant}")
        print(f"  T0_nut_compl: {compositions.T0_nut_compl}")
        print(f"  T0_nut_org: {compositions.T0_nut_org}")
        print(f"  un_tiers_FA_nut_comp: {compositions.un_tiers_FA_nut_comp}")
        print(f"  un_tiers_FA_nut_org: {compositions.un_tiers_FA_nut_org}")
        print(f"  NSc: {compositions.NSc}")
        print(f"  NSC_sc_seq: {compositions.NSC_sc_seq}")
        print(f"  NSC_tps_seq: {compositions.NSC_tps_seq}")
    else:
        print("  Aucune donnée de compositions")

print(f"\n{'='*80}")
print("FIN DE L'AFFICHAGE")
print(f"{'='*80}")

# Fermer la connexion
db.close()

Nombre de fermentations trouvées: 1

FERMENTATION 1

📋 TABLE FERMENTATION:
  Code: F13_2024-05-21
  Souche: EC1118
  Milieu: Chardo PR22
  Volume: Salle
  Csg_T: 16
  Type_fermentation: Valide

📊 TABLE DONNEES_FERMENTATION:
  Nombre de points: 1075
    Point 1: Id=5096445, Temps=0.0, CO2=0.0, V=0.0, Code=F13_2024-05-21
    Point 2: Id=5096446, Temps=0.33, CO2=-0.04, V=0.0, Code=F13_2024-05-21
    Point 3: Id=5096447, Temps=0.67, CO2=-0.06, V=-0.15, Code=F13_2024-05-21
    Point 4: Id=5096448, Temps=1.0, CO2=-0.1, V=-0.096, Code=F13_2024-05-21
    Point 5: Id=5096449, Temps=1.33, CO2=-0.12, V=-0.107, Code=F13_2024-05-21
    Point 6: Id=5096450, Temps=1.67, CO2=-0.14, V=-0.111, Code=F13_2024-05-21
    Point 7: Id=5096451, Temps=2.0, CO2=-0.16, V=-0.099, Code=F13_2024-05-21
    Point 8: Id=5096452, Temps=2.33, CO2=-0.16, V=-0.091, Code=F13_2024-05-21
    Point 9: Id=5096453, Temps=2.67, CO2=-0.16, V=-0.077, Code=F13_2024-05-21
    Point 10: Id=5096454, Temps=3.0, CO2=-0.18, V=-0.064, Code

# *pydantic*

In [23]:
%%writefile schemas.py

from pydantic import BaseModel
from typing import Optional, List

# =============================================================================
# SCHÉMAS CORRESPONDANT AUX MODÈLES UTILISÉS DANS QUERY_HELPER.PY
# =============================================================================

class FermentationBase(BaseModel):
    """Schéma correspondant au modèle models.Fermentation"""
    Code: str
    Souche: Optional[str] = None
    Milieu: Optional[str] = None
    Volume: Optional[str] = None
    Csg_T: Optional[int] = None
    Type_fermentation: Optional[str] = None
    
    class Config:
        from_attributes = True

class DonneesFermentationBase(BaseModel):
    """Schéma correspondant au modèle models.DonneesFermentation"""
    Id: int
    Temps: Optional[float] = None
    CO2: Optional[float] = None
    V: Optional[float] = None
    Code: str
    
    class Config:
        from_attributes = True

class PhaseFermentationBase(BaseModel):
    """Schéma correspondant au modèle models.PhaseFermentation"""
    Code: str
    FIN_LATENCE: Optional[float] = None
    Max_V: Optional[float] = None
    TempsmaxV: Optional[float] = None
    max_co2: Optional[float] = None
    tempsmaxco2: Optional[float] = None
    Maxco2_70: Optional[float] = None
    Maxco2_80: Optional[float] = None
    Maxco2_90: Optional[float] = None
    Maxco2_95: Optional[float] = None
    tempsmaxco2_70: Optional[float] = None
    tempsmaxco2_80: Optional[float] = None
    tempsmaxco2_90: Optional[float] = None
    tempsmaxco2_95: Optional[float] = None
    Fin_fermentation: Optional[float] = None
    
    class Config:
        from_attributes = True

class AnalyseMoutBase(BaseModel):
    """Schéma correspondant au modèle models.AnalyseMout"""
    Code: str
    Synonymes: Optional[str] = None
    Code_Milieu: Optional[str] = None
    Sucres: Optional[str] = None
    Azote: Optional[str] = None
    pH: Optional[str] = None
    Turbidite: Optional[str] = None
    Acide_Malic: Optional[str] = None
    Phytosterols_mg_L: Optional[str] = None
    Ergosterol_mg_L: Optional[str] = None
    SO2_libre: Optional[str] = None
    SO2_Total: Optional[str] = None
    Desaere: Optional[bool] = None
    AT: Optional[str] = None
    phyt_NTU: Optional[str] = None
    ergo_NTU: Optional[str] = None
    Turb_phyto: Optional[str] = None
    
    class Config:
        from_attributes = True

class AnalyseVinBase(BaseModel):
    """Schéma correspondant au modèle models.AnalyseVin"""
    Code: str
    Glucose_fructose_g_l: Optional[str] = None
    Titre_alcoometrique_vol: Optional[str] = None
    pH: Optional[str] = None
    Acidite_volatile_gH2SO4_l: Optional[str] = None
    Acidite_totale_gH2SO4_l: Optional[str] = None
    Dioxyde_soufre_Libre_mg_l: Optional[str] = None
    Dioxyde_soufre_Total_mg_l: Optional[str] = None
    AcideL_malique_g_L: Optional[str] = None
    Nuance_IR: Optional[str] = None
    IC: Optional[str] = None
    
    class Config:
        from_attributes = True

class CompositionsBase(BaseModel):
    """Schéma correspondant au modèle models.Compositions"""
    Code: str
    Code_Souche: Optional[str] = None
    modelisation: Optional[bool] = None
    protectant: Optional[bool] = None
    T0_nut_compl: Optional[bool] = None
    T0_nut_org: Optional[bool] = None
    un_tiers_FA_nut_comp: Optional[bool] = None
    un_tiers_FA_nut_org: Optional[bool] = None
    NSc: Optional[bool] = None
    NSC_sc_seq: Optional[bool] = None
    NSC_tps_seq: Optional[bool] = None
    
    class Config:
        from_attributes = True

# =============================================================================
# SCHÉMA COMPOSITE POUR get_all_data_by_souche_temperature_dict
# =============================================================================

class FermentationCompleteData(BaseModel):
    """
    Schéma correspondant EXACTEMENT à la structure du dictionnaire retourné par 
    get_all_data_by_souche_temperature_dict() dans query_helper.py
    
    Structure exacte du dictionnaire :
    {
        'fermentation': fermentation,
        'donnees_fermentation': donnees,
        'phase_fermentation': phase,
        'analyse_mout': analyse_mout,
        'analyse_vin': analyse_vin,
        'compositions': compositions
    }
    """
    fermentation: FermentationBase
    donnees_fermentation: List[DonneesFermentationBase] = []
    phase_fermentation: Optional[PhaseFermentationBase] = None
    analyse_mout: Optional[AnalyseMoutBase] = None
    analyse_vin: Optional[AnalyseVinBase] = None
    compositions: Optional[CompositionsBase] = None
    
    class Config:
        from_attributes = True


Writing schemas.py


# *main*

In [24]:
%%writefile main.py

from fastapi import FastAPI, Depends, HTTPException, Path, Query
from sqlalchemy.orm import Session
from typing import List, Optional
from database import SessionLocal
import models
import query_helper as helpers
import schemas
from fastapi.responses import Response

app = FastAPI(
    title="Lallemand Oenologie: Fermentation API", 
    description="API pour interroger la base de données Lallemand_oenologie", 
    version="1.0"
)

@app.get("/favicon.ico", include_in_schema=False)
async def favicon():
    return Response(status_code=204)

# =============================================================================
# VALIDATION HELPERS
# =============================================================================

def validate_code(code: str) -> str:
    """Valide et nettoie un code de fermentation."""
    if not code or not code.strip():
        raise HTTPException(status_code=400, detail="Le code ne peut pas être vide")
    
    # Nettoyer le code
    code = code.strip()
    
    # Validation basique (ajustez selon vos règles métier)
    if len(code) > 50:
        raise HTTPException(status_code=400, detail="Le code est trop long (max 50 caractères)")
    
    return code

# =============================================================================
# DEPENDENCY INJECTION
# =============================================================================

def get_db():
    """Crée une session de base de données et la ferme après utilisation."""
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# =============================================================================
# MONITORING & HEALTH CHECKS
# =============================================================================

@app.get(
    "/",
    summary="Health Check - Vérifie si l'API fonctionne",
    description="Point d'entrée pour vérifier si l'API Fermentation est opérationnelle.",
    tags=["Monitoring"],
)
async def health_check():
    """Retourne un message de confirmation que l'API est opérationnelle."""
    return {
        "status": "healthy",
        "message": "API Fermentation Lallemand Oenologie opérationnelle",
        "version": "1.0"
    }

@app.get(
    "/health",
    summary="Health Check détaillé", 
    tags=["Monitoring"],
)
def detailed_health_check(db: Session = Depends(get_db)):
    """Vérifie la connectivité à la base de données."""
    try:
        # Test simple de connexion
        count = db.query(models.Fermentation).count()
        return {
            "status": "healthy",
            "database": "connected", 
            "fermentations_count": count
        }
    except Exception as e:
        raise HTTPException(
            status_code=503, 
            detail=f"Base de données inaccessible: {str(e)}"
        )

# =============================================================================
# FERMENTATIONS - ENDPOINT PRINCIPAL
# =============================================================================

@app.get(
    "/fermentations/{code}",
    summary="Obtenir une fermentation complète par Code",
    description="Retourne une fermentation avec toutes ses données associées : données de fermentation, phases, analyses et compositions.",
    response_model=schemas.FermentationCompleteData,
    tags=["Fermentations"],
    responses={
        404: {"description": "Fermentation non trouvée"},
        400: {"description": "Code invalide"}
    }
)
def get_fermentation_by_code(
    code: str = Path(..., description="Code unique de la fermentation", example="F2024-001"),
    db: Session = Depends(get_db)
):
    """Retourne une fermentation avec toutes ses données associées."""
    try:
        # Valider le code
        validated_code = validate_code(code)
        
        # Récupérer directement les données pour ce code spécifique
        fermentation = helpers.get_fermentation(db, validated_code).first()
        if not fermentation:
            raise HTTPException(
                status_code=404, 
                detail=f"Fermentation {validated_code} non trouvée"
            )
        
        # Récupérer toutes les données associées
        donnees = helpers.get_donnees_fermentation(db, validated_code).all()
        phase = helpers.get_phase_fermentation(db, validated_code).first()
        analyse_mout = helpers.get_analyse_mout(db, validated_code).first()
        analyse_vin = helpers.get_analyse_vin(db, validated_code).first()
        compositions = helpers.get_parametres(db, validated_code).first()
        
        # Construire la réponse
        return schemas.FermentationCompleteData(
            fermentation=schemas.FermentationBase.model_validate(fermentation),
            donnees_fermentation=[
                schemas.DonneesFermentationBase.model_validate(d) 
                for d in donnees
            ],
            phase_fermentation=schemas.PhaseFermentationBase.model_validate(phase) if phase else None,
            analyse_mout=schemas.AnalyseMoutBase.model_validate(analyse_mout) if analyse_mout else None,
            analyse_vin=schemas.AnalyseVinBase.model_validate(analyse_vin) if analyse_vin else None,
            compositions=schemas.CompositionsBase.model_validate(compositions) if compositions else None
        )
        
    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(
            status_code=500, 
            detail=f"Erreur lors de la récupération: {str(e)}"
        )


# =============================================================================
# ENDPOINTS PAR TYPE DE DONNÉES
# =============================================================================

@app.get(
    "/fermentations",
    summary="Liste complète des fermentations",
    description="Retourne la liste des fermentations avec toutes leurs données associées : données de fermentation, phases, analyses et compositions. Filtres optionnels par souche et température.",
    response_model=List[schemas.FermentationCompleteData],
    tags=["Fermentations"],
    responses={
        400: {"description": "Paramètres invalides"}
    }
)
def get_fermentations(
    souche: Optional[str] = Query(None, description="Filtrer par souche", example="EC1118"),
    temperature: Optional[int] = Query(None, description="Filtrer par température", ge=0, le=50),
    db: Session = Depends(get_db)
):
    """Retourne la liste des fermentations avec toutes leurs données associées."""
    try:
        # Validation des paramètres
        if souche is not None:
            souche = souche.strip()
            if not souche:
                raise HTTPException(status_code=400, detail="La souche ne peut pas être vide")
        
        if temperature is not None and (temperature < 0 or temperature > 50):
            raise HTTPException(status_code=400, detail="La température doit être entre 0 et 50°C")
        
        # Utiliser la fonction helper optimisée qui fonctionne parfaitement
        results_raw = helpers.get_all_data_by_souche_temperature_dict(
            db, souche=souche, temperature=temperature
        )
        
        # Convertir directement en schémas Pydantic (même logique que le code fourni)
        results = []
        for result in results_raw:
            fermentation_data = schemas.FermentationCompleteData(
                fermentation=schemas.FermentationBase.model_validate(result['fermentation']),
                donnees_fermentation=[
                    schemas.DonneesFermentationBase.model_validate(d) 
                    for d in result['donnees_fermentation']
                ],
                phase_fermentation=schemas.PhaseFermentationBase.model_validate(result['phase_fermentation']) if result['phase_fermentation'] else None,
                analyse_mout=schemas.AnalyseMoutBase.model_validate(result['analyse_mout']) if result['analyse_mout'] else None,
                analyse_vin=schemas.AnalyseVinBase.model_validate(result['analyse_vin']) if result['analyse_vin'] else None,
                compositions=schemas.CompositionsBase.model_validate(result['compositions']) if result['compositions'] else None
            )
            results.append(fermentation_data)
        
        return results
        
    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(
            status_code=500, 
            detail=f"Erreur lors de la récupération: {str(e)}"
        )

@app.get(
    "/donnees-fermentation/{code}",
    summary="Données de fermentation par code",
    description="Récupère toutes les données temporelles d'une fermentation spécifique (temps, CO2, vitesse).",
    response_model=List[schemas.DonneesFermentationBase],
    tags=["Données"],
    responses={
        404: {"description": "Données non trouvées"},
        400: {"description": "Code invalide"}
    }
)
def get_donnees_fermentation_by_code(
    code: str = Path(..., description="Code de la fermentation", example="F2024-001"),
    db: Session = Depends(get_db)
):
    """Récupère toutes les données d'une fermentation spécifique."""
    try:
        # Valider le code
        validated_code = validate_code(code)
        
        donnees = helpers.get_donnees_fermentation(db, validated_code).all()
        if not donnees:
            raise HTTPException(
                status_code=404,
                detail=f"Aucune donnée trouvée pour la fermentation {validated_code}"
            )
        
        return [schemas.DonneesFermentationBase.model_validate(d) for d in donnees]
    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Erreur lors de la récupération: {str(e)}"
        )

@app.get(
    "/phase-fermentation/{code}",
    summary="Phase de fermentation par code",
    description="Récupère les métriques de phase d'une fermentation (latence, vitesse max, CO2, etc.).",
    response_model=schemas.PhaseFermentationBase,
    tags=["Phases"],
    responses={
        404: {"description": "Phase non trouvée"},
        400: {"description": "Code invalide"}
    }
)
def get_phase_fermentation_by_code(
    code: str = Path(..., description="Code de la fermentation", example="F2024-001"),
    db: Session = Depends(get_db)
):
    """Récupère la phase d'une fermentation spécifique."""
    try:
        # Valider le code
        validated_code = validate_code(code)
        
        phase = helpers.get_phase_fermentation(db, validated_code).first()
        if not phase:
            raise HTTPException(
                status_code=404,
                detail=f"Phase de fermentation {validated_code} non trouvée"
            )
        return schemas.PhaseFermentationBase.model_validate(phase)
    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Erreur lors de la récupération: {str(e)}"
        )

@app.get(
    "/analyse-mout/{code}",
    summary="Analyse de moût par code",
    description="Récupère l'analyse chimique du moût (sucres, azote, pH, turbidité, etc.).",
    response_model=schemas.AnalyseMoutBase,
    tags=["Analyses"],
    responses={
        404: {"description": "Analyse non trouvée"},
        400: {"description": "Code invalide"}
    }
)
def get_analyse_mout_by_code(
    code: str = Path(..., description="Code de la fermentation", example="F2024-001"),
    db: Session = Depends(get_db)
):
    """Récupère l'analyse de moût d'une fermentation spécifique."""
    try:
        # Valider le code
        validated_code = validate_code(code)
        
        analyse = helpers.get_analyse_mout(db, validated_code).first()
        if not analyse:
            raise HTTPException(
                status_code=404,
                detail=f"Analyse de moût {validated_code} non trouvée"
            )
        return schemas.AnalyseMoutBase.model_validate(analyse)
    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Erreur lors de la récupération: {str(e)}"
        )

@app.get(
    "/analyse-vin/{code}",
    summary="Analyse de vin par code",
    description="Récupère l'analyse du vin final (glucose, titre alcoolométrique, acidité, etc.).",
    response_model=schemas.AnalyseVinBase,
    tags=["Analyses"],
    responses={
        404: {"description": "Analyse non trouvée"},
        400: {"description": "Code invalide"}
    }
)
def get_analyse_vin_by_code(
    code: str = Path(..., description="Code de la fermentation", example="F2024-001"),
    db: Session = Depends(get_db)
):
    """Récupère l'analyse de vin d'une fermentation spécifique."""
    try:
        # Valider le code
        validated_code = validate_code(code)
        
        analyse = helpers.get_analyse_vin(db, validated_code).first()
        if not analyse:
            raise HTTPException(
                status_code=404,
                detail=f"Analyse de vin {validated_code} non trouvée"
            )
        return schemas.AnalyseVinBase.model_validate(analyse)
    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Erreur lors de la récupération: {str(e)}"
        )

@app.get(
    "/compositions/{code}",
    summary="Compositions par code",
    description="Récupère les informations sur les compositions et traitements appliqués.",
    response_model=schemas.CompositionsBase,
    tags=["Compositions"],
    responses={
        404: {"description": "Compositions non trouvées"},
        400: {"description": "Code invalide"}
    }
)
def get_compositions_by_code(
    code: str = Path(..., description="Code de la fermentation", example="F2024-001"),
    db: Session = Depends(get_db)
):
    """Récupère les compositions d'une fermentation spécifique."""
    try:
        # Valider le code
        validated_code = validate_code(code)
        
        composition = helpers.get_parametres(db, validated_code).first()
        if not composition:
            raise HTTPException(
                status_code=404,
                detail=f"Compositions {validated_code} non trouvées"
            )
        return schemas.CompositionsBase.model_validate(composition)
    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Erreur lors de la récupération: {str(e)}"
        )


Writing main.py


# *APPLICATION*

In [16]:
%%writefile application.py

import streamlit as st
import requests
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import json

# Configuration de la page
st.set_page_config(
    page_title="Lallemand Œnologie - Fermentation API",
    page_icon="🍷",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Configuration de l'API
API_BASE_URL = st.sidebar.text_input(
    "URL de l'API", 
    value="http://localhost:8000",
    help="URL de base de votre API FastAPI"
)

# Titre principal avec logo
col_logo, col_title = st.columns([1, 4])

with col_logo:
    # Logo Lallemand local
    st.image(
        "logo_lallemand.png",  # Remplacez par le chemin de votre logo
        width=150
    )

with col_title:
    st.title("🍷 Lallemand Œnologie - Dashboard Fermentation")
    
st.markdown("---")

# Sidebar pour la navigation
st.sidebar.title("Navigation")
page = st.sidebar.selectbox(
    "Choisir une fonction",
    ["🏠 Accueil", "🔍 Recherche par Code", "📊 Analyse par Souche/Température", "🩺 Health Check"]
)

# Fonction pour tester la connexion API
def test_api_connection():
    try:
        response = requests.get(f"{API_BASE_URL}/")
        if response.status_code == 200:
            return True, response.json()
        else:
            return False, f"Erreur {response.status_code}: {response.text}"
    except requests.exceptions.RequestException as e:
        return False, f"Erreur de connexion: {str(e)}"

# Fonction pour récupérer les données de fermentation par code
def get_fermentation_data(code):
    try:
        response = requests.get(f"{API_BASE_URL}/fermentation_donnee/{code}")
        if response.status_code == 200:
            return True, response.json()
        else:
            return False, f"Erreur {response.status_code}: {response.text}"
    except requests.exceptions.RequestException as e:
        return False, f"Erreur de connexion: {str(e)}"

# Fonction pour récupérer les données par souche ou température
def get_data_by_filters(souche=None, csg_t=None):
    try:
        params = {}
        if souche:
            params['souche'] = souche
        if csg_t:
            params['csg_t'] = csg_t
        
        response = requests.get(f"{API_BASE_URL}/donnees_by_souche_or_temp", params=params)
        if response.status_code == 200:
            return True, response.json()
        else:
            return False, f"Erreur {response.status_code}: {response.text}"
    except requests.exceptions.RequestException as e:
        return False, f"Erreur de connexion: {str(e)}"

# Page d'accueil
if page == "🏠 Accueil":
    st.header("Bienvenue dans le Dashboard Fermentation")
    
    col1, col2, col3 = st.columns(3)
    
    with col1:
        st.info("🔍 **Recherche par Code**\nRecherchez les données de fermentation spécifiques à un code")
    
    with col2:
        st.info("📊 **Analyse par Souche/Température**\nAnalysez les données en filtrant par souche ou température")
    
    with col3:
        st.info("🩺 **Health Check**\nVérifiez l'état de l'API")
    
    st.markdown("---")
    
    # Test de connexion automatique
    st.subheader("État de la connexion API")
    with st.spinner("Test de connexion..."):
        success, result = test_api_connection()
        if success:
            st.success(f"✅ API connectée: {result.get('message', 'OK')}")
        else:
            st.error(f"❌ Problème de connexion: {result}")

# Page Health Check
elif page == "🩺 Health Check":
    st.header("🩺 Vérification de l'état de l'API")
    
    if st.button("Tester la connexion", type="primary", key="health_check_btn"):
        with st.spinner("Test en cours..."):
            success, result = test_api_connection()
            if success:
                st.success("✅ API opérationnelle")
                st.json(result)
            else:
                st.error(f"❌ {result}")

# Page recherche par code
elif page == "🔍 Recherche par Code":
    st.header("🔍 Recherche de données par Code de fermentation")
    
    code = st.text_input("Entrez le code de fermentation:", placeholder="Ex: FERM001")
    
    if st.button("Rechercher", type="primary", key="search_by_code_btn") and code:
        with st.spinner("Recherche en cours..."):
            success, data = get_fermentation_data(code)
            
            if success:
                st.success(f"✅ Données trouvées pour le code: {code}")
                
                # Affichage des données de fermentation
                if 'fermentation' in data and data['fermentation']:
                    st.subheader("📋 Informations de fermentation")
                    df_fermentation = pd.DataFrame(data['fermentation'])
                    st.dataframe(df_fermentation, use_container_width=True)
                
                # Affichage des données de mesure
                if 'donnee' in data and data['donnee']:
                    st.subheader("📊 Données de mesure")
                    df_donnee = pd.DataFrame(data['donnee'])
                    st.dataframe(df_donnee, use_container_width=True)
                    
                    # Graphiques
                    col1, col2 = st.columns(2)
                    
                    with col1:
                        fig_co2 = px.line(df_donnee, x='Temps', y='CO2', 
                                         title='Évolution du CO2 dans le temps',
                                         labels={'Temps': 'Temps', 'CO2': 'CO2'})
                        st.plotly_chart(fig_co2, use_container_width=True)
                    
                    with col2:
                        fig_v = px.line(df_donnee, x='Temps', y='V', 
                                       title='Évolution du Volume dans le temps',
                                       labels={'Temps': 'Temps', 'V': 'Volume'})
                        st.plotly_chart(fig_v, use_container_width=True)
            else:
                st.error(f"❌ {data}")

# Page analyse par souche/température - VERSION SIMPLIFIÉE
elif page == "📊 Analyse par Souche/Température":
    st.header("📊 Analyse des données par Souche ou Température")
    
    col1, col2 = st.columns(2)
    
    with col1:
        souche = st.text_input("Filtrer par souche:", placeholder="Ex: LEVURE001")
    
    with col2:
        csg_t = st.number_input("Filtrer par température (Csg_T):", min_value=0, step=1, value=None)
    
    # Initialiser session_state pour les données
    if 'filtered_data' not in st.session_state:
        st.session_state.filtered_data = None
    
    # UN SEUL BOUTON ANALYSER
    if st.button("Analyser", type="primary", key="single_analyze_btn"):
        if souche or csg_t:
            with st.spinner("Analyse en cours..."):
                success, data = get_data_by_filters(souche=souche if souche else None, 
                                                  csg_t=int(csg_t) if csg_t else None)
                
                if success and data:
                    st.session_state.filtered_data = {
                        'data': data,
                        'souche': souche,
                        'csg_t': csg_t
                    }
                elif success and not data:
                    st.warning("⚠️ Aucune donnée trouvée pour les critères spécifiés")
                    st.session_state.filtered_data = None
                else:
                    st.error(f"❌ {data}")
                    st.session_state.filtered_data = None
        else:
            st.warning("⚠️ Veuillez spécifier au moins un critère (souche ou température)")
    
    # Affichage des données UNIQUEMENT
    if st.session_state.filtered_data is not None:
        data = st.session_state.filtered_data['data']
        souche_used = st.session_state.filtered_data['souche']
        csg_t_used = st.session_state.filtered_data['csg_t']
        
        # Afficher les critères utilisés
        criteres = []
        if souche_used:
            criteres.append(f"Souche: {souche_used}")
        if csg_t_used:
            criteres.append(f"Température: {csg_t_used}")
        
        st.success(f"✅ {len(data)} enregistrements trouvés - Critères: {', '.join(criteres)}")
        
        df = pd.DataFrame(data)
        
        # UNIQUEMENT LE TABLEAU
        st.subheader("📋 Données filtrées")
        st.dataframe(df, use_container_width=True)
        
        # UNIQUEMENT LE TÉLÉCHARGEMENT
        st.subheader("💾 Téléchargement")
        csv = df.to_csv(index=False)
        st.download_button(
            label="Télécharger les données en CSV",
            data=csv,
            file_name=f'donnees_fermentation_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv',
            mime='text/csv'
        )
        
        # Bouton pour effacer les données
        if st.button("🗑️ Effacer les résultats", key="clear_results_btn"):
            st.session_state.filtered_data = None
            st.rerun()

# Footer
st.markdown("---")
st.markdown("🍷 **Lallemand Œnologie** - Dashboard Fermentation API v0.2")

Overwriting application.py


# *Application combine*

In [17]:
%%writefile combine.py

import streamlit as st
import requests
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import json
import pyodbc
import numpy as np
import os

# Configuration de la page
st.set_page_config(
    page_title="Lallemand Œnologie - Dashboard Fermentation",
    page_icon="🍷",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Fonctions pour l'API (partie originale)
def test_api_connection():
    try:
        response = requests.get(f"{API_BASE_URL}/")
        if response.status_code == 200:
            return True, response.json()
        else:
            return False, f"Erreur {response.status_code}: {response.text}"
    except requests.exceptions.RequestException as e:
        return False, f"Erreur de connexion: {str(e)}"

def get_fermentation_data(code):
    try:
        response = requests.get(f"{API_BASE_URL}/fermentation_donnee/{code}")
        if response.status_code == 200:
            return True, response.json()
        else:
            return False, f"Erreur {response.status_code}: {response.text}"
    except requests.exceptions.RequestException as e:
        return False, f"Erreur de connexion: {str(e)}"

def get_data_by_filters(souche=None, csg_t=None):
    try:
        params = {}
        if souche:
            params['souche'] = souche
        if csg_t:
            params['csg_t'] = csg_t
        
        response = requests.get(f"{API_BASE_URL}/donnees_by_souche_or_temp", params=params)
        if response.status_code == 200:
            return True, response.json()
        else:
            return False, f"Erreur {response.status_code}: {response.text}"
    except requests.exceptions.RequestException as e:
        return False, f"Erreur de connexion: {str(e)}"

# Fonctions pour le traitement de fichiers (nouvelle partie)
def init_session_state():
    """Initialiser les variables de session"""
    if 'folder_path' not in st.session_state:
        st.session_state.folder_path = ''
    if 'folder_files' not in st.session_state:
        st.session_state.folder_files = []
    if 'show_results' not in st.session_state:
        st.session_state.show_results = False
    if 'filtered_data' not in st.session_state:
        st.session_state.filtered_data = None

def get_connection_string(server, database, auth_method, username=None, password=None):
    """Générer la chaîne de connexion selon la méthode d'authentification"""
    if auth_method == "Windows":
        return f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
    else:
        return f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};'

def test_sql_connection(server, database, auth_method, username=None, password=None):
    """Tester la connexion à la base de données SQL Server"""
    try:
        connection_string = get_connection_string(server, database, auth_method, username, password)
        conn = pyodbc.connect(connection_string)
        conn.close()
        return True, "Connexion réussie !"
    except Exception as e:
        return False, f"Erreur de connexion : {str(e)}"

def process_file(file_content, file_name, conn):
    """Traiter un fichier individuel selon la logique originale"""
    try:
        # Créer DataFrame à partir du contenu
        df = pd.DataFrame([x.split('\t') for x in file_content.split('\n') if x])
        separateur = len(df.columns)
        
        cursor = conn.cursor()
        
        # Extraire les informations de base
        Fermenteur = df.loc[:, 0][1].split('=')[1]
        date_experience = df.loc[:, 0][df[df[0].str.startswith('Début')].index[0]].split('=')[1]
        Souche = df.loc[:, 0][df[df[0].str.startswith('Souche')].index[0]].split('=')[1]
        Milieu = df.loc[:, 0][df[df[0].str.startswith('Milieu')].index[0]].split('=')[1]
        
        # Initialiser enreg par défaut
        enreg = 0
        
        if separateur == 13:
            # Traitement pour fermenteur Salle
            volume = "Salle"
            dicti = {'Fermenteur': Fermenteur, 'date_experience': date_experience, 
                    'Souche': Souche, 'Milieu': Milieu, 'volume': volume}
            data = pd.DataFrame(dicti, index=[0])
            data['date_experience'] = pd.to_datetime(data['date_experience'], format='%d/%m/%Y %H:%M')
            data['Code'] = 'F' + data['Fermenteur'].astype(str) + '_' + data['date_experience'].astype(str)
            data['Code'] = data['Code'].str.extract(r'(\w+_\d{4}-\d{2}-\d{2})')
            
            # Traitement des données
            valeurs_a_chercher = ["[Données]", "[Donn�es]"]
            df = df.loc[:,][df.loc[df.loc[:, 0].isin(valeurs_a_chercher)].index[0] + 1:]
            nouvelles_colonnes = df.iloc[0, :len(df.columns)].tolist()
            df.columns = nouvelles_colonnes
            df = df.iloc[1:].reset_index(drop=True)
            
            # Nettoyage des données
            df['Tps inject.'] = df['Tps inject.'].str.replace(',', '.').astype(int)
            max_inject = df['Tps inject.'].max()
            df.drop(columns=['T°C', 'Poids', 'Volume', 'FinPhas', 'Vinst', 'Csg V', 'V5', 'Acc G','Tps inject.'], inplace=True)
            df['Code'] = data['Code'][0]
            df.rename(columns={'Csg T': 'Csg_T'}, inplace=True)
            df['Temps'] = df['Temps'].str.replace(',', '.').astype(float)
            df['Csg_T'] = df['Csg_T'].str.replace(',', '.').astype(float)
            df['CO2'] = df['CO2'].str.replace(',', '.').astype(float)
            df['V11'] = df['V11'].str.replace(',', '.').astype(float)
            df.rename(columns={'V11': 'V'}, inplace=True)
            data['Csg_T'] = int(df['Csg_T'].iloc[0])
            data.drop(columns=['Fermenteur','date_experience'], inplace=True)
            df.drop(columns='Csg_T', inplace=True)
            
            # Traitement des écarts entre deux observations
            taille = len(df)
            compteur = 0
            for i in np.arange(taille-1):
                z = df['Temps'].iloc[i+1] - df['Temps'].iloc[i]
                if z > 70:
                    compteur = compteur + 1
            
            # Traitement des valeurs manquantes
            S = 0
            for i in np.arange(len(df)):
                if df['CO2'].isna().iloc[i] == True:
                    S = S + 1

            H = 0
            for i in np.arange(len(df)):
                if df['V'].isna().iloc[i] == True:
                    H = H + 1
            
            # Nettoyage final
            if len(df) >= 2 and df['CO2'].iloc[-2] > df['CO2'].iloc[-1]:
                df = df[:len(df) - 1]
            
            # Validation des données
            valid_data = True
            for index, row in df.iterrows():
                try:
                    temps = row['Temps']
                    co2 = float(row['CO2']) if pd.notnull(row['CO2']) else None
                    v = float(row['V']) if pd.notnull(row['V']) else None
                    code = row['Code']
                    if co2 is None or v is None:
                        valid_data = False
                        break
                except ValueError as e:
                    valid_data = False
                    break
            
            # Critères de validation pour Salle
            if S==0 and compteur==0 and len(df) > 100 and df['V'].iloc[-1]<0.1 and df['CO2'].max()>60 and df['V'].iloc[12]<0.2 and max_inject==0 and H==0:
                enreg = 1  # Valide
                data['Type_fermentation'] = 'Valide'
            elif len(df) > 50:
                enreg = 2  # Non valide mais à insérer
                data['Type_fermentation'] = 'Non_Valide'
                data['Code'] = data['Code'].astype(str) + '_NO'
                df['Code'] = df['Code'].astype(str) + '_NO'
            else:
                enreg = 0  # À ignorer complètement
            
        else:
            # Traitement pour fermenteur Robot
            volume = "Robot"
            dicti = {'Fermenteur': Fermenteur, 'date_experience': date_experience, 
                    'Souche': Souche, 'Milieu': Milieu, 'volume': volume}
            data = pd.DataFrame(dicti, index=[0])
            data['date_experience'] = pd.to_datetime(data['date_experience'], format="%Y/%m/%d %H:%M")
            data['Code'] = data['Fermenteur'].astype(str) + '_' + data['date_experience'].astype(str)
            data['Code'] = data['Code'].str.extract(r'(\w+_\d{4}-\d{2}-\d{2})')
            
            # Traitement des données
            valeurs_a_chercher = ["[Données]","[Donn�es]"]
            df = df.loc[:,][df.loc[df.loc[:, 0].isin(valeurs_a_chercher)].index[0]+1:]
            nouvelles_colonnes = df.iloc[0, :10].tolist()
            remaining_columns = df.columns[10:].tolist()
            df.columns = nouvelles_colonnes + remaining_columns
            df = df.iloc[1:]
            df = df.iloc[1:].reset_index(drop=True)
            df['Code'] = data['Code'][0]
            df.drop(["Prélèvement","V","V3",'Volume','Poids','T°C'], inplace=True, axis=1)
            df.rename(columns={'Csg T': 'Csg_T'}, inplace=True)
            df.rename(columns={'V5': 'V'}, inplace=True)
            df['Temps'] = df['Temps'].str.replace(',', '.').astype(float)
            df['Csg_T'] = df['Csg_T'].str.replace(',', '.').astype(float)
            df['CO2'] = df['CO2'].str.replace(',', '.').astype(float)
            df['V'] = df['V'].str.replace(',', '.').astype(float)
            data['Csg_T'] = int(df['Csg_T'].iloc[0])
            data.drop(columns=['Fermenteur','date_experience'], inplace=True)
            df.drop(columns='Csg_T', inplace=True)
            
            # Traitement des écarts
            taille = len(df)
            compteur = 0
            for i in np.arange(taille-1):
                z = df['Temps'].iloc[i+1] - df['Temps'].iloc[i]
                if z > 70:
                    compteur = compteur + 1
            
            # Comptage des lignes complètement vides
            z = 0
            for i in np.arange(len(df)):
                if df['V'].isna().iloc[i]==True and df['CO2'].isna().iloc[i]==True and df['Temps'].isna().iloc[i]==True:
                    z = z + 1
            
            # Nettoyage final
            if len(df) >= 2 and df['CO2'].iloc[-2] > df['CO2'].iloc[-1]:
                df = df[:len(df) - 1]
            
            # Validation des données
            valid_data = True
            for index, row in df.iterrows():
                try:
                    temps = row['Temps']
                    co2 = float(row['CO2']) if pd.notnull(row['CO2']) else None
                    v = float(row['V']) if pd.notnull(row['V']) else None
                    code = row['Code']
                    if co2 is None or v is None:
                        valid_data = False
                        break
                except ValueError as e:
                    valid_data = False
                    break
            
            # Critères de validation pour Robot
            if z==0 and compteur==0 and len(df) > 50 and df['CO2'].max()>60 and df['V'].iloc[-1]<0.1 and df['V'].iloc[4]<0.2:
                enreg = 1  # Valide
                data['Type_fermentation'] = 'Valide'
            elif len(df) > 15:
                enreg = 2  # Non valide mais à insérer
                data['Type_fermentation'] = 'Non_Valide'
                data['Code'] = data['Code'].astype(str) + '_NO'
                df['Code'] = df['Code'].astype(str) + '_NO'
            else:
                enreg = 0  # À ignorer complètement
                        
        # Vérifier si le code existe déjà
        cursor.execute("SELECT COUNT(*) FROM Fermentation WHERE Code = ?", data['Code'][0])
        if cursor.fetchone()[0] > 0:
            cursor.close()
            return False, f"Code {data['Code'][0]} existe déjà dans la base"
        
        if (enreg == 1 or enreg == 2) and valid_data:
            # Insérer dans Fermentation (valides ET non valides)
            for index, row in data.iterrows():
                cursor.execute('''
                    INSERT INTO Fermentation (Souche, Milieu, volume, Code, Csg_T, Type_fermentation)
                    VALUES (?, ?, ?, ?, ?, ?)
                ''', row['Souche'], row['Milieu'], row['volume'], row['Code'], row['Csg_T'], row['Type_fermentation'])
            
            # Valider la transaction après Fermentation
            conn.commit()
            
            # Insérer dans donnee (id est auto-increment, donc pas besoin de l'inclure)
            nb_points = len(df)
            for index, row in df.iterrows():
                cursor.execute('''
                    INSERT INTO donnee (Temps, CO2, V, Code)
                    VALUES (?, ?, ?, ?)
                ''', row['Temps'], row['CO2'], row['V'], row['Code'])
            
            # Valider la transaction après donnee
            conn.commit()
            
            # Fermer le curseur
            cursor.close()
            
            # Message de succès détaillé selon le type
            if enreg == 1:
                success_msg = f"✅ DONNÉES VALIDES INSÉRÉES !\n"
                success_msg += f"📊 {nb_points} points de données ajoutés\n"
                success_msg += f"🏷️ Code: {data['Code'][0]}\n"
                success_msg += f"🧪 Souche: {data['Souche'][0]}\n"
                success_msg += f"🌾 Milieu: {data['Milieu'][0]}\n"
                success_msg += f"⚗️ Volume: {data['volume'][0]}\n"
                success_msg += f"🔬 Type: {data['Type_fermentation'][0]} ✅"
            else:  # enreg == 2
                success_msg = f"⚠️ DONNÉES NON VALIDES INSÉRÉES\n"
                success_msg += f"📊 {nb_points} points de données ajoutés\n"
                success_msg += f"🏷️ Code: {data['Code'][0]} (avec suffixe _NO)\n"
                success_msg += f"🧪 Souche: {data['Souche'][0]}\n"
                success_msg += f"🌾 Milieu: {data['Milieu'][0]}\n"
                success_msg += f"⚗️ Volume: {data['volume'][0]}\n"
                success_msg += f"🔬 Type: {data['Type_fermentation'][0]} ⚠️"
            
            return True, success_msg
        elif enreg == 0:
            cursor.close()
            return False, f"📊 Fichier {file_name} ignoré (pas assez de données)"
        else:
            cursor.close()
            return False, f"❌ Données invalides détectées dans {file_name}"
            
    except Exception as e:
        return False, f"❌ Erreur lors du traitement : {str(e)}"

def process_folder_files(server, database, auth_method, username=None, password=None):
    """Traiter tous les fichiers d'un dossier"""
    if not st.session_state.folder_files:
        st.error("❌ Aucun fichier à traiter")
        return
    
    # Test de connexion avant traitement
    success, message = test_sql_connection(server, database, auth_method, username, password)
    if not success:
        st.error(f"Impossible de se connecter à la base : {message}")
        return
    
    # Connexion à la base
    try:
        connection_string = get_connection_string(server, database, auth_method, username, password)
        conn = pyodbc.connect(connection_string)
        
        # Progress bar et statistiques
        progress_bar = st.progress(0)
        status_text = st.empty()
        
        fichiers_traites = 0
        fichiers_valides = 0
        fichiers_non_valides = 0
        fichiers_erreur = 0
        fichiers_duplicate = 0
        fichiers_ignores = 0
        
        # Traitement des fichiers
        total_files = len(st.session_state.folder_files)
        
        for i, file_path in enumerate(st.session_state.folder_files):
            # Mise à jour de la progress bar
            progress = (i + 1) / total_files
            progress_bar.progress(progress)
            file_name = os.path.basename(file_path)
            status_text.text(f"📂 Traitement en cours : {file_name} ({i+1}/{total_files})")
            
            try:
                # Lire le fichier avec différents encodages
                encodages = ['utf-8', 'latin-1', 'cp1252', 'ascii', 'utf-16', 'utf-32']
                content = None
                
                for encodage in encodages:
                    try:
                        with open(file_path, "r", encoding=encodage) as fichier:
                            content = fichier.read()
                        break
                    except UnicodeDecodeError:
                        continue
                
                if content is None:
                    fichiers_erreur += 1
                    continue
                
                # Traiter le fichier
                success, result_message = process_file(content, file_name, conn)
                
                if success:
                    fichiers_traites += 1
                    if "VALIDES INSÉRÉES" in result_message:
                        fichiers_valides += 1
                    elif "NON VALIDES INSÉRÉES" in result_message:
                        fichiers_non_valides += 1
                else:
                    if "existe déjà" in result_message:
                        fichiers_duplicate += 1
                    elif "ignoré" in result_message:
                        fichiers_ignores += 1
                    else:
                        fichiers_erreur += 1
                            
            except Exception as e:
                fichiers_erreur += 1
        
        # Fermer la connexion
        conn.close()
        
        # Résumé final
        progress_bar.progress(1.0)
        status_text.text("✅ Traitement terminé !")
        
        # Marquer que les résultats sont prêts à afficher
        st.session_state.show_results = True
        st.session_state.results_data = {
            'total_files': total_files,
            'fichiers_traites': fichiers_traites,
            'fichiers_valides': fichiers_valides,
            'fichiers_non_valides': fichiers_non_valides,
            'fichiers_duplicate': fichiers_duplicate,
            'fichiers_ignores': fichiers_ignores,
            'fichiers_erreur': fichiers_erreur
        }
        
        # Vider la liste des fichiers traités
        st.session_state.folder_files = []
        
    except Exception as e:
        st.error(f"❌ Erreur lors de la connexion à la base : {str(e)}")

# Initialisation
init_session_state()

# Configuration dans la sidebar - DÉFINI EN PREMIER
st.sidebar.title("🔧 Configuration")

# Choix du mode de fonctionnement - DÉFINI EN PREMIER
mode = st.sidebar.radio(
    "Mode de fonctionnement :",
    ["🌐 API (FastAPI)", "🗄️ Base SQL Server"],
    help="API: pour consulter les données via FastAPI. SQL Server: pour traiter et insérer des fichiers"
)

# Titre principal avec logo - APRÈS LA DÉFINITION DE MODE
col_logo, col_title = st.columns([1, 4])

with col_logo:
    # Logo Lallemand local
    try:
        st.image("logo_lallemand.png", width=150)
    except:
        st.markdown("**🍷 LALLEMAND**")

with col_title:
    if mode == "🌐 API (FastAPI)":
        st.title("🍷 Lallemand Œnologie - Dashboard Fermentation")
    else:
        st.title("🍷 Lallemand Œnologie - Traitement des fichiers Text")
    
st.markdown("---")

# Configuration dans la sidebar

if mode == "🌐 API (FastAPI)":
    # Configuration API
    API_BASE_URL = st.sidebar.text_input(
        "URL de l'API", 
        value="http://localhost:8000",
        help="URL de base de votre API FastAPI"
    )
    
    # Sidebar pour la navigation
    st.sidebar.title("Navigation")
    page = st.sidebar.selectbox(
        "Choisir une fonction",
        ["🏠 Accueil", "🔍 Recherche par Code", "📊 Analyse par Souche/Température", "🩺 Health Check"]
    )

else:
    # Configuration SQL Server
    st.sidebar.subheader("🗄️ Base de données")
    
    server = st.sidebar.text_input(
        "Serveur SQL :",
        value="localhost\\SQLEXPRESS",
        help="Exemple: localhost\\SQLEXPRESS ou IP:PORT"
    )
    
    database = st.sidebar.text_input(
        "Base de données :",
        value="Lallemand_oenologie",
        help="Nom de votre base de données"
    )
    
    # Méthode d'authentification
    auth_method = st.sidebar.radio(
        "Authentification :",
        ["Windows", "SQL Server"],
        help="Windows: utilise vos identifiants Windows. SQL Server: utilise un nom d'utilisateur/mot de passe"
    )
    
    username = None
    password = None
    
    if auth_method == "SQL Server":
        username = st.sidebar.text_input("Nom d'utilisateur :")
        password = st.sidebar.text_input("Mot de passe :", type="password")
    
    # Test de connexion
    if st.sidebar.button("🔌 Tester la connexion"):
        success, message = test_sql_connection(server, database, auth_method, username, password)
        if success:
            st.sidebar.success(message)
        else:
            st.sidebar.error(message)
    
    page = "📁 Traitement de Fichiers"

# Pages selon le mode
if mode == "🌐 API (FastAPI)":
    
    # Page d'accueil
    if page == "🏠 Accueil":
        st.header("Bienvenue dans le Dashboard Fermentation")
        
        col1, col2, col3 = st.columns(3)
        
        with col1:
            st.info("🔍 **Recherche par Code**\nRecherchez les données de fermentation spécifiques à un code")
        
        with col2:
            st.info("📊 **Analyse par Souche/Température**\nAnalysez les données en filtrant par souche ou température")
        
        with col3:
            st.info("🩺 **Health Check**\nVérifiez l'état de l'API")
        
        st.markdown("---")
        
        # Test de connexion automatique
        st.subheader("État de la connexion API")
        with st.spinner("Test de connexion..."):
            success, result = test_api_connection()
            if success:
                st.success(f"✅ API connectée: {result.get('message', 'OK')}")
            else:
                st.error(f"❌ Problème de connexion: {result}")

    # Page Health Check
    elif page == "🩺 Health Check":
        st.header("🩺 Vérification de l'état de l'API")
        
        if st.button("Tester la connexion", type="primary", key="health_check_btn"):
            with st.spinner("Test en cours..."):
                success, result = test_api_connection()
                if success:
                    st.success("✅ API opérationnelle")
                    st.json(result)
                else:
                    st.error(f"❌ {result}")

    # Page recherche par code
    elif page == "🔍 Recherche par Code":
        st.header("🔍 Recherche de données par Code de fermentation")
        
        code = st.text_input("Entrez le code de fermentation:", placeholder="Ex: FERM001")
        
        if st.button("Rechercher", type="primary", key="search_by_code_btn") and code:
            with st.spinner("Recherche en cours..."):
                success, data = get_fermentation_data(code)
                
                if success:
                    st.success(f"✅ Données trouvées pour le code: {code}")
                    
                    # Affichage des données de fermentation
                    if 'fermentation' in data and data['fermentation']:
                        st.subheader("📋 Informations de fermentation")
                        df_fermentation = pd.DataFrame(data['fermentation'])
                        st.dataframe(df_fermentation, use_container_width=True)
                    
                    # Affichage des données de mesure
                    if 'donnee' in data and data['donnee']:
                        st.subheader("📊 Données de mesure")
                        df_donnee = pd.DataFrame(data['donnee'])
                        st.dataframe(df_donnee, use_container_width=True)
                        
                        # Graphiques
                        col1, col2 = st.columns(2)
                        
                        with col1:
                            fig_co2 = px.line(df_donnee, x='Temps', y='CO2', 
                                             title='Évolution du CO2 dans le temps',
                                             labels={'Temps': 'Temps', 'CO2': 'CO2'})
                            st.plotly_chart(fig_co2, use_container_width=True)
                        
                        with col2:
                            fig_v = px.line(df_donnee, x='Temps', y='V', 
                                           title='Évolution du Volume dans le temps',
                                           labels={'Temps': 'Temps', 'V': 'Volume'})
                            st.plotly_chart(fig_v, use_container_width=True)
                else:
                    st.error(f"❌ {data}")

    # Page analyse par souche/température
    elif page == "📊 Analyse par Souche/Température":
        
        col1, col2 = st.columns(2)
        
        with col1:
            souche = st.text_input("Filtrer par souche:", placeholder="Ex: LEVURE001")
        
        with col2:
            csg_t = st.number_input("Filtrer par température (Csg_T):", min_value=0, step=1, value=None)
        
        # UN SEUL BOUTON ANALYSER
        if st.button("Analyser", type="primary", key="single_analyze_btn"):
            if souche or csg_t:
                with st.spinner("Analyse en cours..."):
                    success, data = get_data_by_filters(souche=souche if souche else None, 
                                                      csg_t=int(csg_t) if csg_t else None)
                    
                    if success and data:
                        st.session_state.filtered_data = {
                            'data': data,
                            'souche': souche,
                            'csg_t': csg_t
                        }
                    elif success and not data:
                        st.warning("⚠️ Aucune donnée trouvée pour les critères spécifiés")
                        st.session_state.filtered_data = None
                    else:
                        st.error(f"❌ {data}")
                        st.session_state.filtered_data = None
            else:
                st.warning("⚠️ Veuillez spécifier au moins un critère (souche ou température)")
        
        # Affichage des données UNIQUEMENT
        if st.session_state.filtered_data is not None:
            data = st.session_state.filtered_data['data']
            souche_used = st.session_state.filtered_data['souche']
            csg_t_used = st.session_state.filtered_data['csg_t']
            
            # Afficher les critères utilisés
            criteres = []
            if souche_used:
                criteres.append(f"Souche: {souche_used}")
            if csg_t_used:
                criteres.append(f"Température: {csg_t_used}")
            
            st.success(f"✅ {len(data)} enregistrements trouvés - Critères: {', '.join(criteres)}")
            
            df = pd.DataFrame(data)
            
            # UNIQUEMENT LE TABLEAU
            st.subheader("📋 Données filtrées")
            st.dataframe(df, use_container_width=True)
            
            # UNIQUEMENT LE TÉLÉCHARGEMENT
            st.subheader("💾 Téléchargement")
            csv = df.to_csv(index=False)
            st.download_button(
                label="Télécharger les données en CSV",
                data=csv,
                file_name=f'donnees_fermentation_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv',
                mime='text/csv'
            )
            
            # Bouton pour effacer les données
            if st.button("🗑️ Effacer les résultats", key="clear_results_btn"):
                st.session_state.filtered_data = None
                st.rerun()

else:
    # MODE SQL SERVER - TRAITEMENT DE FICHIERS
    st.header("📁 Traitement des Données de Fermentation")
    
    # SEUL CHOIX : Dossier local
    st.subheader("📁 Sélectionner un dossier")
    
    # Chemin manuel
    folder_path = st.text_input(
        "Chemin du dossier :",
        value=st.session_state.get('folder_path', ''),
        placeholder="C:\\Users\\VotreNom\\Documents\\Fermentation",
        help="Entrez le chemin complet du dossier contenant vos fichiers .txt"
    )
    
    # Sauvegarder dans session state
    if folder_path != st.session_state.get('folder_path', ''):
        st.session_state.folder_path = folder_path
    
    # Validation et navigation du dossier
    if folder_path:
        if os.path.exists(folder_path) and os.path.isdir(folder_path):
            st.success(f"✅ Dossier trouvé : {folder_path}")
            
            # Afficher le contenu du dossier
            try:
                files_in_dir = os.listdir(folder_path)
                txt_files = [f for f in files_in_dir if f.endswith('.txt')]
                
                if txt_files:
                    st.info(f"📊 {len(txt_files)} fichier(s) .txt trouvé(s)")
                    
                    # Aperçu des fichiers
                    with st.expander(f"👁️ Aperçu des fichiers ({len(txt_files)} fichiers)"):
                        for i, file in enumerate(txt_files[:10]):  # Afficher max 10 fichiers
                            st.write(f"• {file}")
                        if len(txt_files) > 10:
                            st.write(f"... et {len(txt_files) - 10} autres fichiers")
                else:
                    st.warning("⚠️ Aucun fichier .txt trouvé dans ce dossier")
                    
                # Navigation vers le dossier parent
                parent_dir = os.path.dirname(folder_path)
                if parent_dir != folder_path:  # Éviter la boucle infinie à la racine
                    if st.button(f"⬆️ Dossier parent: {os.path.basename(parent_dir)}"):
                        st.session_state.folder_path = parent_dir
                        st.success(f"📁 Dossier parent sélectionné. Cliquez sur 'Scanner le dossier' pour rafraîchir.")
                
                # Afficher les sous-dossiers
                subdirs = [d for d in files_in_dir if os.path.isdir(os.path.join(folder_path, d))]
                if subdirs:
                    st.write("**Sous-dossiers disponibles :**")
                    cols = st.columns(min(3, len(subdirs)))
                    for i, subdir in enumerate(subdirs[:6]):  # Max 6 sous-dossiers
                        with cols[i % 3]:
                            if st.button(f"📁 {subdir}", key=f"subdir_{i}"):
                                st.session_state.folder_path = os.path.join(folder_path, subdir)
                                st.success(f"📁 Sous-dossier '{subdir}' sélectionné. Cliquez sur 'Scanner le dossier' pour rafraîchir.")
                                
            except PermissionError:
                st.error("❌ Accès refusé à ce dossier")
            except Exception as e:
                st.error(f"❌ Erreur lors de la lecture du dossier : {str(e)}")
                
        elif folder_path.strip():  # Si un chemin est entré mais invalide
            st.error("❌ Dossier non trouvé ou inaccessible")
    
    # Scanner le dossier
    if st.button("🔍 Scanner le dossier", disabled=not (folder_path and os.path.exists(folder_path))):
        try:
            fichiers = []
            for fichier in os.listdir(folder_path):
                if fichier.endswith('.txt'):
                    fichiers.append(os.path.join(folder_path, fichier))
            
            if fichiers:
                st.session_state.folder_files = fichiers
                st.success(f"✅ {len(fichiers)} fichier(s) .txt détecté(s) et prêt(s) à traiter")
            else:
                st.warning("⚠️ Aucun fichier .txt trouvé dans ce dossier")
                st.session_state.folder_files = []
                
        except Exception as e:
            st.error(f"❌ Erreur lors du scan : {str(e)}")
    
    # Afficher les fichiers trouvés précédemment
    if st.session_state.folder_files:
        st.info(f"📁 Prêt à traiter {len(st.session_state.folder_files)} fichier(s)")
        
        if st.button("🚀 Traiter les fichiers du dossier", key="process_folder"):
            process_folder_files(server, database, auth_method, username, password)
    
    # Afficher les résultats après traitement
    if st.session_state.get('show_results', False):
        results = st.session_state.results_data
        
        # Statistiques finales
        st.markdown("---")
        st.subheader("📊 Résumé du traitement")
        
        col1, col2, col3, col4, col5, col6 = st.columns(6)
        with col1:
            st.metric("📁 Total", results['total_files'])
        with col2:
            st.metric("✅ Valides", results['fichiers_valides'], delta_color="normal")
        with col3:
            st.metric("⚠️ Non valides", results['fichiers_non_valides'], delta_color="off")
        with col4:
            st.metric("🔄 Doublons", results['fichiers_duplicate'], delta_color="off")
        with col5:
            st.metric("📊 Ignorés", results['fichiers_ignores'], delta_color="off")
        with col6:
            st.metric("❌ Erreurs", results['fichiers_erreur'], delta_color="off")
        
        # Messages détaillés
        total_inserted = results['fichiers_valides'] + results['fichiers_non_valides']
        if total_inserted > 0:
            st.balloons()
            
            if results['fichiers_valides'] > 0 and results['fichiers_non_valides'] > 0:
                st.success(f"🎉 Traitement terminé ! {results['fichiers_valides']} fichier(s) valides et {results['fichiers_non_valides']} fichier(s) non valides ajoutés à la base.")
            elif results['fichiers_valides'] > 0:
                st.success(f"🎉 Traitement terminé avec succès ! {results['fichiers_valides']} fichier(s) valides ajoutés à la base.")
            else:
                st.warning(f"⚠️ Traitement terminé. {results['fichiers_non_valides']} fichier(s) non valides ajoutés à la base.")
        
        # Alertes spécifiques
        if results['fichiers_ignores'] > 0:
            st.info(f"ℹ️ {results['fichiers_ignores']} fichier(s) ignoré(s) (pas assez de données)")
        
        if results['fichiers_duplicate'] > 0:
            st.warning(f"🔄 {results['fichiers_duplicate']} fichier(s) déjà présent(s) dans la base")
        
        if results['fichiers_erreur'] > 0:
            st.error(f"❌ {results['fichiers_erreur']} fichier(s) en erreur")
        
        # Bouton pour traiter un autre dossier
        st.markdown("---")
        if st.button("📁 Traiter un autre dossier", key="new_folder_button"):
            # Réinitialiser complètement tout le formulaire
            st.session_state.folder_path = ''
            st.session_state.folder_files = []
            st.session_state.show_results = False
            if 'results_data' in st.session_state:
                del st.session_state.results_data
            # Forcer le rechargement complet de la page
            st.success("✅ Formulaire réinitialisé ! Sélectionnez un nouveau dossier.")
            st.rerun()

    # Aide dans un expander en bas
    with st.expander("💡 Aide et informations"):
        col1, col2 = st.columns(2)
        
        with col1:
            st.markdown("""
            **🔧 Prérequis :**
            - Tables **Fermentation** et **donnee** dans SQL Server
            - ODBC Driver 17 for SQL Server installé
            - Fichiers .txt avec format spécifique
            
            **📋 Structure Fermentation :**
            - Souche, Milieu, volume, Code, Csg_T, Type_fermentation
            
            **📋 Structure donnee :**
            - id (auto), Temps, CO2, V, Code (FK)
            """)
        
        with col2:
            st.markdown("""
            **🎯 Types de validation :**
            - ✅ **Valides** : Respectent tous les critères
            - ⚠️ **Non valides** : Conservées avec suffixe _NO
            - 📊 **Ignorées** : Pas assez de données
            
            **🔍 Critères Salle vs Robot :**
            - **Salle** : >100 pts (valide), >50 pts (non valide)
            - **Robot** : >50 pts (valide), >15 pts (non valide)
            """)

# Footer
st.markdown("---")
st.markdown("🍷 **Lallemand Œnologie** - Dashboard Fermentation Combiné v1.0")

Overwriting combine.py


In [23]:
%%writefile code.py


import tkinter as tk
from tkinter import ttk, filedialog, messagebox, scrolledtext
import pandas as pd
import numpy as np
import pyodbc
import os
import requests
import json
from datetime import datetime
import threading
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTk
from matplotlib.figure import Figure

class LallemandFermentationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("🍷 Lallemand Œnologie - Application Fermentation")
        self.root.geometry("1200x800")
        self.root.configure(bg='#f0f0f0')
        
        # Variables
        self.mode = tk.StringVar(value="API")
        self.folder_path = tk.StringVar()
        self.folder_files = []
        self.results_data = {}
        
        # Configuration API
        self.api_url = tk.StringVar(value="http://localhost:8000")
        
        # Configuration SQL
        self.server = tk.StringVar(value="localhost\\SQLEXPRESS")
        self.database = tk.StringVar(value="Lallemand_oenologie")
        self.auth_method = tk.StringVar(value="Windows")
        self.username = tk.StringVar()
        self.password = tk.StringVar()
        
        self.setup_ui()
        
    def setup_ui(self):
        """Configurer l'interface utilisateur"""
        # Style
        style = ttk.Style()
        style.theme_use('clam')
        
        # Frame principal
        main_frame = ttk.Frame(self.root)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # En-tête avec logo et titre
        header_frame = ttk.Frame(main_frame)
        header_frame.pack(fill=tk.X, pady=(0, 10))
        
        # Logo (simulation)
        logo_frame = ttk.Frame(header_frame, relief='raised', borderwidth=2)
        logo_frame.pack(side=tk.LEFT, padx=(0, 20))
        ttk.Label(logo_frame, text="🍷\nLALLEMAND\nŒNOLOGIE", 
                 font=('Arial', 10, 'bold'), background='white').pack(padx=10, pady=5)
        
        # Titre principal
        self.title_label = ttk.Label(header_frame, text="Dashboard Fermentation", 
                                    font=('Arial', 16, 'bold'))
        self.title_label.pack(side=tk.LEFT, anchor='w')
        
        # Séparateur
        ttk.Separator(main_frame, orient='horizontal').pack(fill=tk.X, pady=5)
        
        # Frame pour mode et configuration
        config_main_frame = ttk.Frame(main_frame)
        config_main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Sidebar (à gauche)
        sidebar = ttk.LabelFrame(config_main_frame, text="🔧 Configuration", padding=10)
        sidebar.pack(side=tk.LEFT, fill=tk.Y, padx=(0, 10))
        sidebar.configure(width=300)
        
        # Contenu principal (à droite)
        self.content_frame = ttk.Frame(config_main_frame)
        self.content_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)
        
        self.setup_sidebar(sidebar)
        self.setup_content()
        
    def setup_sidebar(self, parent):
        """Configurer la sidebar"""
        # Choix du mode
        mode_frame = ttk.LabelFrame(parent, text="Mode de fonctionnement", padding=5)
        mode_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Radiobutton(mode_frame, text="🌐 API (FastAPI)", variable=self.mode, 
                       value="API", command=self.on_mode_change).pack(anchor='w')
        ttk.Radiobutton(mode_frame, text="🗄️ Base SQL Server", variable=self.mode, 
                       value="SQL", command=self.on_mode_change).pack(anchor='w')
        
        # Configuration API
        self.api_config_frame = ttk.LabelFrame(parent, text="Configuration API", padding=5)
        self.api_config_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Label(self.api_config_frame, text="URL API:").pack(anchor='w')
        ttk.Entry(self.api_config_frame, textvariable=self.api_url, width=30).pack(fill=tk.X, pady=(0, 5))
        ttk.Button(self.api_config_frame, text="🔌 Tester Connexion", 
                  command=self.test_api_connection).pack(fill=tk.X)
        
        # Configuration SQL
        self.sql_config_frame = ttk.LabelFrame(parent, text="Configuration SQL Server", padding=5)
        self.sql_config_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Label(self.sql_config_frame, text="Serveur:").pack(anchor='w')
        ttk.Entry(self.sql_config_frame, textvariable=self.server, width=30).pack(fill=tk.X, pady=(0, 5))
        
        ttk.Label(self.sql_config_frame, text="Base de données:").pack(anchor='w')
        ttk.Entry(self.sql_config_frame, textvariable=self.database, width=30).pack(fill=tk.X, pady=(0, 5))
        
        ttk.Label(self.sql_config_frame, text="Authentification:").pack(anchor='w')
        auth_frame = ttk.Frame(self.sql_config_frame)
        auth_frame.pack(fill=tk.X, pady=(0, 5))
        ttk.Radiobutton(auth_frame, text="Windows", variable=self.auth_method, 
                       value="Windows").pack(side=tk.LEFT)
        ttk.Radiobutton(auth_frame, text="SQL Server", variable=self.auth_method, 
                       value="SQL").pack(side=tk.LEFT)
        
        ttk.Label(self.sql_config_frame, text="Utilisateur:").pack(anchor='w')
        self.username_entry = ttk.Entry(self.sql_config_frame, textvariable=self.username, width=30)
        self.username_entry.pack(fill=tk.X, pady=(0, 5))
        
        ttk.Label(self.sql_config_frame, text="Mot de passe:").pack(anchor='w')
        self.password_entry = ttk.Entry(self.sql_config_frame, textvariable=self.password, 
                                       show="*", width=30)
        self.password_entry.pack(fill=tk.X, pady=(0, 5))
        
        ttk.Button(self.sql_config_frame, text="🔌 Tester Connexion", 
                  command=self.test_sql_connection).pack(fill=tk.X)
        
        # Navigation API
        self.api_nav_frame = ttk.LabelFrame(parent, text="Navigation", padding=5)
        self.api_nav_frame.pack(fill=tk.X, pady=(0, 10))
        
        self.nav_var = tk.StringVar(value="Accueil")
        nav_options = ["🏠 Accueil", "🔍 Recherche par Code", 
                      "📊 Analyse Souche/Temp", "🩺 Health Check"]
        
        for option in nav_options:
            ttk.Radiobutton(self.api_nav_frame, text=option, variable=self.nav_var, 
                           value=option, command=self.on_nav_change).pack(anchor='w')
        
        # Aide
        help_frame = ttk.LabelFrame(parent, text="💡 Aide", padding=5)
        help_frame.pack(fill=tk.X, expand=True)
        
        help_text = scrolledtext.ScrolledText(help_frame, height=8, width=30, wrap=tk.WORD)
        help_text.pack(fill=tk.BOTH, expand=True)
        
        help_content = """🎯 UTILISATION:

Mode API:
• Consulter les données existantes
• Rechercher par code
• Analyser par souche/température

Mode SQL Server:
• Traiter des fichiers .txt
• Insérer en base de données
• Validation automatique

📋 PRÉREQUIS:
• Tables Fermentation et donnee
• ODBC Driver 17 for SQL Server
• Fichiers .txt formatés"""
        
        help_text.insert(tk.END, help_content)
        help_text.config(state=tk.DISABLED)
        
        self.on_mode_change()  # Initialiser l'affichage
        
    def setup_content(self):
        """Configurer la zone de contenu principal"""
        # Notebook pour les différentes pages
        self.notebook = ttk.Notebook(self.content_frame)
        self.notebook.pack(fill=tk.BOTH, expand=True)
        
        # Pages API
        self.setup_api_pages()
        
        # Pages SQL
        self.setup_sql_pages()
        
    def setup_api_pages(self):
        """Configurer les pages du mode API"""
        # Page Accueil
        self.home_frame = ttk.Frame(self.notebook)
        self.notebook.add(self.home_frame, text="🏠 Accueil")
        
        ttk.Label(self.home_frame, text="Bienvenue dans le Dashboard Fermentation", 
                 font=('Arial', 14, 'bold')).pack(pady=20)
        
        info_frame = ttk.Frame(self.home_frame)
        info_frame.pack(fill=tk.BOTH, expand=True, padx=20)
        
        for i, (title, desc) in enumerate([
            ("🔍 Recherche par Code", "Recherchez les données de fermentation spécifiques"),
            ("📊 Analyse Souche/Temp", "Analysez les données par souche ou température"),
            ("🩺 Health Check", "Vérifiez l'état de l'API")
        ]):
            frame = ttk.LabelFrame(info_frame, text=title, padding=10)
            frame.pack(fill=tk.X, pady=10)
            ttk.Label(frame, text=desc).pack()
        
        # Page Recherche
        self.search_frame = ttk.Frame(self.notebook)
        self.notebook.add(self.search_frame, text="🔍 Recherche")
        
        search_input_frame = ttk.Frame(self.search_frame)
        search_input_frame.pack(fill=tk.X, padx=20, pady=10)
        
        ttk.Label(search_input_frame, text="Code de fermentation:").pack(side=tk.LEFT)
        self.search_code = tk.StringVar()
        ttk.Entry(search_input_frame, textvariable=self.search_code, width=20).pack(side=tk.LEFT, padx=10)
        ttk.Button(search_input_frame, text="🔍 Rechercher", 
                  command=self.search_by_code).pack(side=tk.LEFT)
        
        # Zone d'affichage des résultats
        self.search_results_frame = ttk.Frame(self.search_frame)
        self.search_results_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
        # Page Analyse
        self.analysis_frame = ttk.Frame(self.notebook)
        self.notebook.add(self.analysis_frame, text="📊 Analyse")
        
        filter_frame = ttk.Frame(self.analysis_frame)
        filter_frame.pack(fill=tk.X, padx=20, pady=10)
        
        ttk.Label(filter_frame, text="Souche:").grid(row=0, column=0, sticky='w', padx=5)
        self.filter_souche = tk.StringVar()
        ttk.Entry(filter_frame, textvariable=self.filter_souche, width=20).grid(row=0, column=1, padx=5)
        
        ttk.Label(filter_frame, text="Température:").grid(row=0, column=2, sticky='w', padx=5)
        self.filter_temp = tk.StringVar()
        ttk.Entry(filter_frame, textvariable=self.filter_temp, width=10).grid(row=0, column=3, padx=5)
        
        ttk.Button(filter_frame, text="📊 Analyser", 
                  command=self.analyze_data).grid(row=0, column=4, padx=10)
        
        # Zone d'affichage des analyses
        self.analysis_results_frame = ttk.Frame(self.analysis_frame)
        self.analysis_results_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
        # Page Health Check
        self.health_frame = ttk.Frame(self.notebook)
        self.notebook.add(self.health_frame, text="🩺 Health")
        
        ttk.Button(self.health_frame, text="🩺 Vérifier l'API", 
                  command=self.health_check).pack(pady=20)
        
        self.health_results = scrolledtext.ScrolledText(self.health_frame, height=15, width=80)
        self.health_results.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
    def setup_sql_pages(self):
        """Configurer les pages du mode SQL"""
        # Page Traitement de fichiers
        self.files_frame = ttk.Frame(self.notebook)
        self.notebook.add(self.files_frame, text="📁 Traitement")
        
        # Sélection de dossier
        folder_frame = ttk.LabelFrame(self.files_frame, text="📁 Sélection de dossier", padding=10)
        folder_frame.pack(fill=tk.X, padx=20, pady=10)
        
        path_frame = ttk.Frame(folder_frame)
        path_frame.pack(fill=tk.X)
        
        ttk.Label(path_frame, text="Dossier:").pack(side=tk.LEFT)
        ttk.Entry(path_frame, textvariable=self.folder_path, width=50).pack(side=tk.LEFT, padx=10, fill=tk.X, expand=True)
        ttk.Button(path_frame, text="📂 Parcourir", command=self.browse_folder).pack(side=tk.RIGHT)
        
        buttons_frame = ttk.Frame(folder_frame)
        buttons_frame.pack(fill=tk.X, pady=10)
        
        ttk.Button(buttons_frame, text="🔍 Scanner Dossier", 
                  command=self.scan_folder).pack(side=tk.LEFT, padx=5)
        ttk.Button(buttons_frame, text="🚀 Traiter Fichiers", 
                  command=self.process_files).pack(side=tk.LEFT, padx=5)
        
        # Zone d'affichage des fichiers
        files_list_frame = ttk.LabelFrame(self.files_frame, text="📄 Fichiers détectés", padding=10)
        files_list_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
        self.files_listbox = tk.Listbox(files_list_frame, height=10)
        files_scrollbar = ttk.Scrollbar(files_list_frame, orient=tk.VERTICAL, command=self.files_listbox.yview)
        self.files_listbox.configure(yscrollcommand=files_scrollbar.set)
        
        self.files_listbox.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        files_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        # Zone de progression et résultats
        self.progress_frame = ttk.LabelFrame(self.files_frame, text="📊 Progression", padding=10)
        self.progress_frame.pack(fill=tk.X, padx=20, pady=10)
        
        self.progress_bar = ttk.Progressbar(self.progress_frame, mode='determinate')
        self.progress_bar.pack(fill=tk.X, pady=5)
        
        self.progress_label = ttk.Label(self.progress_frame, text="Prêt à traiter...")
        self.progress_label.pack()
        
        # Résultats
        self.results_text = scrolledtext.ScrolledText(self.progress_frame, height=8, width=80)
        self.results_text.pack(fill=tk.BOTH, expand=True, pady=10)
        
    def on_mode_change(self):
        """Gérer le changement de mode"""
        mode = self.mode.get()
        
        if mode == "API":
            self.title_label.config(text="Dashboard Fermentation")
            self.api_config_frame.pack(fill=tk.X, pady=(0, 10))
            self.api_nav_frame.pack(fill=tk.X, pady=(0, 10))
            self.sql_config_frame.pack_forget()
            
            # Vérifier si notebook existe avant de l'utiliser
            if hasattr(self, 'notebook'):
                # Afficher les pages API
                for i in range(4):  # 4 premières pages = API
                    self.notebook.tab(i, state="normal")
                for i in range(4, self.notebook.index("end")):  # Pages SQL
                    self.notebook.tab(i, state="hidden")
                
        else:  # SQL
            self.title_label.config(text="Traitement des fichiers Text")
            self.sql_config_frame.pack(fill=tk.X, pady=(0, 10))
            self.api_config_frame.pack_forget()
            self.api_nav_frame.pack_forget()
            
            # Vérifier si notebook existe avant de l'utiliser
            if hasattr(self, 'notebook'):
                # Afficher les pages SQL
                for i in range(4):  # Pages API
                    self.notebook.tab(i, state="hidden")
                for i in range(4, self.notebook.index("end")):  # Pages SQL
                    self.notebook.tab(i, state="normal")
                
                # Sélectionner la première page SQL visible
                self.notebook.select(4)
    
    def on_nav_change(self):
        """Gérer le changement de navigation API"""
        nav = self.nav_var.get()
        page_map = {
            "🏠 Accueil": 0,
            "🔍 Recherche par Code": 1,
            "📊 Analyse Souche/Temp": 2,
            "🩺 Health Check": 3
        }
        if nav in page_map:
            self.notebook.select(page_map[nav])
    
    def test_api_connection(self):
        """Tester la connexion API"""
        def test():
            try:
                response = requests.get(f"{self.api_url.get()}/", timeout=5)
                if response.status_code == 200:
                    messagebox.showinfo("Connexion", "✅ API connectée avec succès!")
                else:
                    messagebox.showerror("Erreur", f"❌ Erreur {response.status_code}")
            except Exception as e:
                messagebox.showerror("Erreur", f"❌ Erreur de connexion: {str(e)}")
        
        threading.Thread(target=test, daemon=True).start()
    
    def test_sql_connection(self):
        """Tester la connexion SQL Server"""
        def test():
            try:
                if self.auth_method.get() == "Windows":
                    conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={self.server.get()};DATABASE={self.database.get()};Trusted_Connection=yes;'
                else:
                    conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={self.server.get()};DATABASE={self.database.get()};UID={self.username.get()};PWD={self.password.get()};'
                
                conn = pyodbc.connect(conn_str)
                conn.close()
                messagebox.showinfo("Connexion", "✅ Connexion SQL réussie!")
            except Exception as e:
                messagebox.showerror("Erreur", f"❌ Erreur SQL: {str(e)}")
        
        threading.Thread(target=test, daemon=True).start()
    
    def search_by_code(self):
        """Rechercher par code de fermentation"""
        code = self.search_code.get().strip()
        if not code:
            messagebox.showwarning("Attention", "Veuillez entrer un code de fermentation")
            return
        
        def search():
            try:
                response = requests.get(f"{self.api_url.get()}/fermentation_donnee/{code}")
                if response.status_code == 200:
                    data = response.json()
                    self.display_search_results(data)
                else:
                    messagebox.showerror("Erreur", f"❌ Code non trouvé: {response.status_code}")
            except Exception as e:
                messagebox.showerror("Erreur", f"❌ Erreur de recherche: {str(e)}")
        
        threading.Thread(target=search, daemon=True).start()
    
    def display_search_results(self, data):
        """Afficher les résultats de recherche"""
        # Nettoyer la zone d'affichage
        for widget in self.search_results_frame.winfo_children():
            widget.destroy()
        
        if 'fermentation' in data and data['fermentation']:
            # Informations de fermentation
            info_frame = ttk.LabelFrame(self.search_results_frame, text="📋 Informations", padding=10)
            info_frame.pack(fill=tk.X, pady=5)
            
            fermentation = data['fermentation'][0]
            info_text = f"Souche: {fermentation.get('Souche', 'N/A')} | Milieu: {fermentation.get('Milieu', 'N/A')} | Volume: {fermentation.get('volume', 'N/A')} | Température: {fermentation.get('Csg_T', 'N/A')}°C"
            ttk.Label(info_frame, text=info_text).pack()
        
        if 'donnee' in data and data['donnee']:
            # Graphiques
            graph_frame = ttk.LabelFrame(self.search_results_frame, text="📊 Graphiques", padding=10)
            graph_frame.pack(fill=tk.BOTH, expand=True, pady=5)
            
            df = pd.DataFrame(data['donnee'])
            
            fig = Figure(figsize=(12, 4), dpi=100)
            
            # CO2
            ax1 = fig.add_subplot(121)
            ax1.plot(df['Temps'], df['CO2'], 'b-', linewidth=2)
            ax1.set_title('Évolution du CO2')
            ax1.set_xlabel('Temps')
            ax1.set_ylabel('CO2')
            ax1.grid(True, alpha=0.3)
            
            # Volume
            ax2 = fig.add_subplot(122)
            ax2.plot(df['Temps'], df['V'], 'r-', linewidth=2)
            ax2.set_title('Évolution du Volume')
            ax2.set_xlabel('Temps')
            ax2.set_ylabel('Volume')
            ax2.grid(True, alpha=0.3)
            
            fig.tight_layout()
            
            canvas = FigureCanvasTk(fig, graph_frame)
            canvas.draw()
            canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
    
    def analyze_data(self):
        """Analyser les données par souche/température"""
        souche = self.filter_souche.get().strip() or None
        temp = self.filter_temp.get().strip()
        temp = int(temp) if temp.isdigit() else None
        
        if not souche and not temp:
            messagebox.showwarning("Attention", "Veuillez spécifier au moins un critère")
            return
        
        def analyze():
            try:
                params = {}
                if souche:
                    params['souche'] = souche
                if temp:
                    params['csg_t'] = temp
                
                response = requests.get(f"{self.api_url.get()}/donnees_by_souche_or_temp", params=params)
                if response.status_code == 200:
                    data = response.json()
                    self.display_analysis_results(data)
                else:
                    messagebox.showerror("Erreur", f"❌ Erreur: {response.status_code}")
            except Exception as e:
                messagebox.showerror("Erreur", f"❌ Erreur d'analyse: {str(e)}")
        
        threading.Thread(target=analyze, daemon=True).start()
    
    def display_analysis_results(self, data):
        """Afficher les résultats d'analyse"""
        # Nettoyer la zone d'affichage
        for widget in self.analysis_results_frame.winfo_children():
            widget.destroy()
        
        if not data:
            ttk.Label(self.analysis_results_frame, text="⚠️ Aucune donnée trouvée").pack(pady=20)
            return
        
        # Statistiques
        stats_frame = ttk.LabelFrame(self.analysis_results_frame, text="📊 Statistiques", padding=10)
        stats_frame.pack(fill=tk.X, pady=5)
        
        ttk.Label(stats_frame, text=f"✅ {len(data)} enregistrements trouvés").pack()
        
        # Tableau des données (limité aux 100 premiers)
        table_frame = ttk.LabelFrame(self.analysis_results_frame, text="📋 Données (100 premiers)", padding=10)
        table_frame.pack(fill=tk.BOTH, expand=True, pady=5)
        
        # Créer un Treeview pour le tableau
        columns = list(data[0].keys()) if data else []
        tree = ttk.Treeview(table_frame, columns=columns, show='headings', height=15)
        
        # Configurer les colonnes
        for col in columns:
            tree.heading(col, text=col)
            tree.column(col, width=100, anchor='center')
        
        # Ajouter les données (100 premiers)
        for i, row in enumerate(data[:100]):
            tree.insert('', 'end', values=[row.get(col, '') for col in columns])
        
        # Scrollbars
        v_scrollbar = ttk.Scrollbar(table_frame, orient=tk.VERTICAL, command=tree.yview)
        h_scrollbar = ttk.Scrollbar(table_frame, orient=tk.HORIZONTAL, command=tree.xview)
        tree.configure(yscrollcommand=v_scrollbar.set, xscrollcommand=h_scrollbar.set)
        
        tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        v_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        h_scrollbar.pack(side=tk.BOTTOM, fill=tk.X)
        
        # Bouton d'export
        export_frame = ttk.Frame(self.analysis_results_frame)
        export_frame.pack(fill=tk.X, pady=5)
        
        ttk.Button(export_frame, text="💾 Exporter en CSV", 
                  command=lambda: self.export_to_csv(data)).pack()
    
    def export_to_csv(self, data):
        """Exporter les données en CSV"""
        if not data:
            return
        
        filename = filedialog.asksaveasfilename(
            defaultextension=".csv",
            filetypes=[("CSV files", "*.csv"), ("All files", "*.*")],
            title="Sauvegarder les données"
        )
        
        if filename:
            try:
                df = pd.DataFrame(data)
                df.to_csv(filename, index=False)
                messagebox.showinfo("Export", f"✅ Données exportées vers {filename}")
            except Exception as e:
                messagebox.showerror("Erreur", f"❌ Erreur d'export: {str(e)}")
    
    def health_check(self):
        """Vérifier l'état de l'API"""
        def check():
            self.health_results.delete(1.0, tk.END)
            self.health_results.insert(tk.END, "🔍 Vérification en cours...\n\n")
            
            try:
                response = requests.get(f"{self.api_url.get()}/", timeout=5)
                if response.status_code == 200:
                    result = response.json()
                    self.health_results.insert(tk.END, "✅ API OPÉRATIONNELLE\n\n")
                    self.health_results.insert(tk.END, f"Réponse: {json.dumps(result, indent=2)}")
                else:
                    self.health_results.insert(tk.END, f"❌ ERREUR {response.status_code}\n\n")
                    self.health_results.insert(tk.END, response.text)
            except Exception as e:
                self.health_results.insert(tk.END, f"❌ ERREUR DE CONNEXION\n\n{str(e)}")
        
        threading.Thread(target=check, daemon=True).start()
    
    def browse_folder(self):
        """Parcourir et sélectionner un dossier"""
        folder = filedialog.askdirectory(title="Sélectionner le dossier contenant les fichiers .txt")
        if folder:
            self.folder_path.set(folder)
    
    def scan_folder(self):
        """Scanner le dossier pour les fichiers .txt"""
        folder = self.folder_path.get()
        if not folder or not os.path.exists(folder):
            messagebox.showerror("Erreur", "❌ Dossier non trouvé")
            return
        
        try:
            self.folder_files = []
            self.files_listbox.delete(0, tk.END)
            
            for file in os.listdir(folder):
                if file.endswith('.txt'):
                    file_path = os.path.join(folder, file)
                    self.folder_files.append(file_path)
                    self.files_listbox.insert(tk.END, file)
            
            if self.folder_files:
                messagebox.showinfo("Scan", f"✅ {len(self.folder_files)} fichier(s) .txt trouvé(s)")
            else:
                messagebox.showwarning("Scan", "⚠️ Aucun fichier .txt trouvé")
                
        except Exception as e:
            messagebox.showerror("Erreur", f"❌ Erreur de scan: {str(e)}")
    
    def process_files(self):
        """Traiter les fichiers du dossier"""
        if not self.folder_files:
            messagebox.showwarning("Attention", "⚠️ Aucun fichier à traiter")
            return
        
        def process():
            try:
                # Test de connexion
                if self.auth_method.get() == "Windows":
                    conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={self.server.get()};DATABASE={self.database.get()};Trusted_Connection=yes;'
                else:
                    conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={self.server.get()};DATABASE={self.database.get()};UID={self.username.get()};PWD={self.password.get()};'
                
                conn = pyodbc.connect(conn_str)
                
                # Initialiser les compteurs
                total_files = len(self.folder_files)
                processed = valides = non_valides = erreurs = duplicates = ignores = 0
                
                self.progress_bar['maximum'] = total_files
                self.results_text.delete(1.0, tk.END)
                self.results_text.insert(tk.END, f"🚀 Traitement de {total_files} fichiers...\n\n")
                
                for i, file_path in enumerate(self.folder_files):
                    file_name = os.path.basename(file_path)
                    self.progress_label.config(text=f"Traitement: {file_name} ({i+1}/{total_files})")
                    self.progress_bar['value'] = i + 1
                    self.root.update()
                    
                    try:
                        # Simuler le traitement (ici vous mettriez la vraie logique)
                        with open(file_path, 'r', encoding='utf-8') as f:
                            content = f.read()
                        
                        # Simulation du traitement
                        if "Fermenteur" in content and "Souche" in content:
                            processed += 1
                            valides += 1
                            self.results_text.insert(tk.END, f"✅ {file_name} - Valide\n")
                        else:
                            erreurs += 1
                            self.results_text.insert(tk.END, f"❌ {file_name} - Erreur format\n")
                            
                    except Exception as e:
                        erreurs += 1
                        self.results_text.insert(tk.END, f"❌ {file_name} - Erreur: {str(e)}\n")
                    
                    self.results_text.see(tk.END)
                
                conn.close()
                
                # Résumé final
                self.progress_label.config(text="✅ Traitement terminé!")
                self.results_text.insert(tk.END, f"\n📊 RÉSUMÉ:\n")
                self.results_text.insert(tk.END, f"Total: {total_files}\n")
                self.results_text.insert(tk.END, f"✅ Valides: {valides}\n")
                self.results_text.insert(tk.END, f"⚠️ Non valides: {non_valides}\n")
                self.results_text.insert(tk.END, f"❌ Erreurs: {erreurs}\n")
                
                if processed > 0:
                    messagebox.showinfo("Succès", f"🎉 Traitement terminé!\n{processed} fichier(s) traités avec succès")
                
            except Exception as e:
                messagebox.showerror("Erreur", f"❌ Erreur de traitement: {str(e)}")
        
        threading.Thread(target=process, daemon=True).start()

def main():
    root = tk.Tk()
    app = LallemandFermentationApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()


Overwriting code.py


In [20]:
pip install tkinter

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter
